In [24]:
'''DEF d'un namespace = un dossier dans une base

index: veille-strategique
NAMESPACES (isolation par type de données) :
├── namespace: financial_reports
│   └── SEC Edgar, SIRENE, rapports d'entreprises
├── namespace: news
│   └── NewsAPI, Google News RSS, communiqués de presse
├── namespace: macro_data
│   └── FRED, yfinance, DBnomics, Adzuna
├── namespace: social_signals
│   └── Bluesky, Reddit (futur), Twitter (futur)
├── namespace: web_quarantine
│   └── Recherches web dynamiques (RAG agentic)
├── namespace: startups
│   └── bluesky, crunchbase ...
└── namespace: facts
    └── Facts structurés (KPIs, deals, market size)

BUT : Retrieval ciblé, Éviter le bruit, Prompting plus intelligent (« Réponds uniquement à partir des données issues du namespace financial_reports et cite les sources »)Ça réduit les hallucinations, Scalabilité propre :

-Tu peux ajouter de nouvelles sources sans casser l’existant

-Tu peux purger un namespace sans toucher aux autres

-Tu peux tester des sources expérimentales (social_signals) sans polluer le corpus principal

, 
1 index = 1 projet RAG
N namespaces = N types de sources
'''

"DEF d'un namespace = un dossier dans une base\n\nindex: veille-strategique\nNAMESPACES (isolation par type de données) :\n├── namespace: financial_reports\n│   └── SEC Edgar, SIRENE, rapports d'entreprises\n├── namespace: news\n│   └── NewsAPI, Google News RSS, communiqués de presse\n├── namespace: macro_data\n│   └── FRED, yfinance, DBnomics, Adzuna\n├── namespace: social_signals\n│   └── Bluesky, Reddit (futur), Twitter (futur)\n├── namespace: web_quarantine\n│   └── Recherches web dynamiques (RAG agentic)\n├── namespace: startups\n│   └── bluesky, crunchbase ...\n└── namespace: facts\n    └── Facts structurés (KPIs, deals, market size)\n\nBUT : Retrieval ciblé, Éviter le bruit, Prompting plus intelligent (« Réponds uniquement à partir des données issues du namespace financial_reports et cite les sources »)Ça réduit les hallucinations, Scalabilité propre :\n\n-Tu peux ajouter de nouvelles sources sans casser l’existant\n\n-Tu peux purger un namespace sans toucher aux autres\n\n-Tu p

In [25]:
# Clean l'index ( ATTENTION A NE PAS EXECUTER A CHAQUZ FOIS !!)


'''
from pinecone import Pinecone
import os
from dotenv import load_dotenv

load_dotenv() 

api_key=os.getenv("PINECONE_API_KEY")
pc = Pinecone(api_key)
INDEX_NAME = "kpmg-veille"

# Supprimer TOUS les vecteurs de TOUS les namespaces
index = pc.Index(INDEX_NAME)
stats = index.describe_index_stats()

for namespace in stats.namespaces.keys():
    print(f" Suppression du namespace '{namespace}'...")
    index.delete(delete_all=True, namespace=namespace)

print("Index nettoyé")

'''


Index nettoyé


In [26]:
"""
NOTEBOOK 1 : Configuration et Nettoyage Pinecone
================================================

OBJECTIF : Réinitialiser complètement l'environnement vectoriel
           et créer une architecture propre avec namespaces.

RÉFÉRENCES :
- Pinecone Docs : https://docs.pinecone.io/docs/python-client
- LangChain Pinecone : https://python.langchain.com/docs/integrations/vectorstores/pinecone

MÉTHODOLOGIE :
1. Supprimer l'index existant (stratégie Option A validée)
2. Recréer un index optimisé pour Mistral embeddings (dimension 1024)
3. Valider la structure des namespaces
"""

import os
import time
from dotenv import load_dotenv
from pinecone import Pinecone, ServerlessSpec

# ═══════════════════════════════════════════════════════════════
# SECTION 1 : CHARGEMENT DES VARIABLES D'ENVIRONNEMENT
# ═══════════════════════════════════════════════════════════════

load_dotenv()

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
#PINECONE_ENVIRONMENT = os.getenv("PINECONE_ENVIRONMENT", "us-east-1")

if not PINECONE_API_KEY:
    raise ValueError(" PINECONE_API_KEY manquante dans .env")

print(" Variables d'environnement chargées")

# ═══════════════════════════════════════════════════════════════
# SECTION 2 : INITIALISATION CLIENT PINECONE
# ═══════════════════════════════════════════════════════════════

"""
JUSTIFICATION : 
Pinecone v3+ utilise une nouvelle API avec ServerlessSpec.
Cela permet une scalabilité automatique sans gérer de pods.

Référence : https://docs.pinecone.io/docs/new-api
"""

pc = Pinecone(api_key=PINECONE_API_KEY)

print("Client Pinecone initialisé")


# NE PAS EXECUTER, UNIQUEMENT SI ON VEUT REPARTIR DE 0.

# ═══════════════════════════════════════════════════════════════
# SECTION 3 : SUPPRESSION DE L'INDEX EXISTANT
# ═══════════════════════════════════════════════════════════════

""""
 Suppression totale

POURQUOI ?
- Garantit un environnement propre sans données parasites
- Évite les conflits de dimension d'embeddings
- Permet de repartir sur des métadonnées structurées
"""

'''
INDEX_NAME = "kpmg-veille"


def clean_pinecone_index():
    """Supprime l'index existant s'il existe"""
    try:
        existing_indexes = [idx.name for idx in pc.list_indexes()]
        
        if INDEX_NAME in existing_indexes:
            print(f" Index '{INDEX_NAME}' détecté. Suppression en cours...")
            pc.delete_index(INDEX_NAME)
            
            # Attendre la suppression complète (bonnes pratiques Pinecone)
            while INDEX_NAME in [idx.name for idx in pc.list_indexes()]:
                print("    Attente de la suppression...")
                time.sleep(2)
            
            print(" Index supprimé avec succès")
        else:
            print(f" Aucun index '{INDEX_NAME}' trouvé (normal si 1ère exécution)")
    
    except Exception as e:
        print(f" Erreur lors du nettoyage : {e}")
        raise

clean_pinecone_index()


# ═══════════════════════════════════════════════════════════════
# SECTION 4 : CRÉATION DU NOUVEL INDEX
# ═══════════════════════════════════════════════════════════════

"""
CONFIGURATION OPTIMALE POUR MISTRAL EMBEDDINGS

1. DIMENSION : 1024
   - Mistral-embed génère des vecteurs de 1024 dimensions
   - Référence : https://docs.mistral.ai/capabilities/embeddings/

2. MÉTRIQUE : cosine
   - Standard pour la similarité sémantique
   - Recommandée par LangChain pour les embeddings textuels
   - Référence : https://python.langchain.com/docs/modules/data_connection/vectorstores/

3. SERVERLESS SPEC :
   - Cloud 'aws', Région 'us-east-1'
   - Scalabilité automatique (critique pour la veille en production)
   - Pas de gestion de pods
"""


def create_optimized_index():
    """Crée un index Pinecone optimisé pour la veille stratégique"""
    try:
        print(f" Création de l'index '{INDEX_NAME}'...")
        
        pc.create_index(
            name=INDEX_NAME,
            dimension=1024,  # Dimension Mistral-embed
            metric="cosine",  # Similarité sémantique
            spec=ServerlessSpec(
                cloud="aws",
                region="us-east-1"  # Utiliser votre région Pinecone
            )
        )
        
        # Attendre que l'index soit prêt
        while not pc.describe_index(INDEX_NAME).status['ready']:
            print("    Initialisation de l'index...")
            time.sleep(2)
        
        print(" Index créé et opérationnel")
        
    except Exception as e:
        print(f" Erreur lors de la création : {e}")
        raise

create_optimized_index()


# ═══════════════════════════════════════════════════════════════
# SECTION 5 : VALIDATION DE LA STRUCTURE
# ═══════════════════════════════════════════════════════════════

"""
ARCHITECTURE DES NAMESPACES

Chaque namespace correspond à un type de source de données :
- financial_reports : SEC EDGAR, rapports annuels
- news : NewsAPI, communiqués de presse
- startups : Crunchbase (futur)
- macro_data : yfinance, données économiques
- social_signals : Reddit, Twitter (futur)

AVANTAGES :
✓ Isolation des sources pour des requêtes ciblées
✓ Possibilité de filtrer par namespace lors du retrieval
✓ Gestion indépendante du cycle de vie des données
✓ Facilite le debugging et les audits
"""


NAMESPACES = [
    "financial_reports",  # SEC EDGAR, SIRENE
    "news",               # NewsAPI, RSS, Press Releases
    "macro_data",         # yfinance, FRED, DBnomics, Adzuna
    "startups",           # Crunchbase (À IMPLÉMENTER)
    "social_signals",     # Bluesky (+ Reddit/Twitter futur)
    "web_quarantine",     # Recherches web RAG agentic
    "facts"              # Facts structurés FACT-RAG
]

def validate_index_structure():
    """Affiche la configuration de l'index"""
    try:
        index_stats = pc.Index(INDEX_NAME).describe_index_stats()
        
        print("\n" + "="*60)
        print(" CONFIGURATION DE L'INDEX")
        print("="*60)
        print(f"Nom : {INDEX_NAME}")
        print(f"Dimension : 1024 (Mistral-embed)")
        print(f"Métrique : cosine")
        print(f"Vecteurs totaux : {index_stats.get('total_vector_count', 0)}")
        print(f"\n Namespaces définis :")
        for ns in NAMESPACES:
            print(f"   - {ns}")
        print("="*60 + "\n")
        
        print(" Index prêt pour l'ingestion de données")
        
    except Exception as e:
        print(f" Erreur de validation : {e}")
        raise

validate_index_structure()

'''
# ═══════════════════════════════════════════════════════════════
# SECTION 6 : CHECKLIST DE VALIDATION
# ═══════════════════════════════════════════════════════════════

print("\n CHECKLIST COMPLÉTÉE :")
print("   ☑ Index existant supprimé")
print("   ☑ Nouvel index créé avec dimension 1024")
print("   ☑ Métrique cosine configurée")
print("   ☑ Serverless spec activé")
print("   ☑ Namespaces documentés")
print("\n Prêt pour l'ingestion (Notebook 2)")



 Variables d'environnement chargées
Client Pinecone initialisé
 Index 'kpmg-veille' détecté. Suppression en cours...
 Index supprimé avec succès
 Création de l'index 'kpmg-veille'...
 Index créé et opérationnel

 CONFIGURATION DE L'INDEX
Nom : kpmg-veille
Dimension : 1024 (Mistral-embed)
Métrique : cosine
Vecteurs totaux : 0

 Namespaces définis :
   - financial_reports
   - news
   - macro_data
   - startups
   - social_signals
   - web_quarantine
   - facts

 Index prêt pour l'ingestion de données

 CHECKLIST COMPLÉTÉE :
   ☑ Index existant supprimé
   ☑ Nouvel index créé avec dimension 1024
   ☑ Métrique cosine configurée
   ☑ Serverless spec activé
   ☑ Namespaces documentés

 Prêt pour l'ingestion (Notebook 2)


In [27]:
"""
NOTEBOOK 2 : Ingestion Multi-Sources
====================================

OBJECTIF : Créer un pipeline robuste d'ingestion de données
           depuis SEC EDGAR, NewsAPI, communiqués de presse et yfinance.

RÉFÉRENCES :
- LangChain Document Loaders : https://python.langchain.com/docs/modules/data_connection/document_loaders/
- SEC EDGAR : https://www.sec.gov/edgar/sec-api-documentation
- NewsAPI : https://newsapi.org/docs
- yfinance : https://pypi.org/project/yfinance/

ARCHITECTURE :
1. Loaders modulaires par source
2. Métadonnées riches (source, date, type)
3. Gestion d'erreurs et retry
4. Logging pour traçabilité
"""
"""
NOTEBOOK 2 : Ingestion Multi-Sources (VERSION CORRIGÉE)
=======================================================

"""

# ═══════════════════════════════════════════════════════════════
# IMPORTS 
# ═══════════════════════════════════════════════════════════════

import os
import json
import time
import random 
from datetime import datetime, timedelta
from typing import List, Dict
from dotenv import load_dotenv

# Web Scraping & Parsing
from bs4 import BeautifulSoup
import requests
import feedparser

# LangChain
from langchain_core.documents import Document
from langchain_community.document_loaders import WebBaseLoader

# APIs Financières & Économiques
import yfinance as yf
import pandas as pd

import dbnomics as db

from atproto import Client

load_dotenv()

# ═══════════════════════════════════════════════════════════════
# CONFIGURATION
# ═══════════════════════════════════════════════════════════════

NEWSAPI_KEY = os.getenv("NEWSAPI_KEY")
NEWSAPI_ENDPOINT = "https://newsapi.org/v2/everything"

SEC_USER_AGENT = os.getenv("SEC_USER_AGENT", "Student research@education.com")
SEC_BASE_URL = "https://data.sec.gov/submissions/"

FRED_API_KEY = os.getenv("FRED_API_KEY")
FRED_BASE_URL = os.getenv("FRED_BASE_URL")

PAPPERS_API_KEY = os.getenv("PAPPERS_API_KEY")
PAPPERS_BASE_URL = os.getenv("PAPPERS_BASE_URL")

API_KEY = "7618440f-8b06-444b-9844-0f8b06144b2e"
BASE_URL = "https://api.insee.fr/api-sirene/3.11"

ADZUNA_APP_ID= os.getenv("ADZUNA_APP_ID")
ADZUNA_APP_KEY= os.getenv("ADZUNA_APP_KEY")

BLUESKY_HANDLE= os.getenv("BLUESKY_HANDLE")
BLUESKY_APP_PASSWORD= os.getenv("BLUESKY_APP_PASSWORD")

LOGS_DIR = "ingestion_logs"
os.makedirs(LOGS_DIR, exist_ok=True)


def log_ingestion(source: str, status: str, details: str):
    """Log avec timestamp"""
    timestamp = datetime.now().isoformat()
    log_entry = f"[{timestamp}] {source} - {status} : {details}\n"
    
    with open(f"{LOGS_DIR}/ingestion.log", "a") as f:
        f.write(log_entry)
    
    print(log_entry.strip())

print(" Configuration des sources initialisée\n")

from config import KPMG_DBNOMICS_SERIES, ALL_BLUESKY_QUERIES


# ═══════════════════════════════════════════════════════════════
# SOURCE 1 : SEC EDGAR - METADATA
# ═══════════════════════════════════════════════════════════════

def load_sec_edgar_filing(cik: str, filing_type: str = "10-K", limit: int = 5) -> List[Document]:
    """Charge les dépôts SEC (limité aux N plus récents)"""
    try:
        headers = {"User-Agent": SEC_USER_AGENT}
        url = f"{SEC_BASE_URL}CIK{cik.zfill(10)}.json"
        
        log_ingestion("SEC_EDGAR", "INFO", f"Requête pour CIK {cik}")
        
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        
        data = response.json()
        company_name = data.get("name", "Unknown")
        filings = data.get("filings", {}).get("recent", {})
        
        documents = []
        count = 0
        
        for i, form in enumerate(filings.get("form", [])):
            if form == filing_type and count < limit:
                filing_date = filings["filingDate"][i]
                accession = filings["accessionNumber"][i]
                primary_doc = filings["primaryDocument"][i]
                
                acc_no_formatted = accession.replace("-", "")
                doc_url = f"https://www.sec.gov/Archives/edgar/data/{cik}/{acc_no_formatted}/{primary_doc}"
                
                doc = Document(
                    page_content=f"SEC Filing {filing_type} for {company_name} on {filing_date}. "
                                f"This regulatory filing provides financial information and disclosures. "
                                f"Accession Number: {accession}",
                    metadata={
                        "source": "sec_edgar",
                        "company": company_name,
                        "cik": cik,
                        "filing_type": filing_type,
                        "filing_date": filing_date,
                        "accession_number": accession,
                        "url": doc_url,
                        "namespace": "financial_reports"
                    }
                )
                documents.append(doc)
                count += 1
        
        log_ingestion("SEC_EDGAR", "SUCCESS", f"{len(documents)} documents trouvés pour {company_name}")
        return documents
    
    except Exception as e:
        log_ingestion("SEC_EDGAR", "ERROR", str(e))
        return []


# ═══════════════════════════════════════════════════════════════
# SEC EDGAR - DOCUMENTS COMPLETS (Version Optimisée : 1 Doc / Rapport)
# ═══════════════════════════════════════════════════════════════

def load_sec_full_document(filing_url: str, filing_metadata: dict) -> List[Document]:
    """
    Télécharge et compresse un rapport SEC complet en UN SEUL document JSON.
    """
    try:
        log_ingestion("SEC_EDGAR_FULL", "INFO", f"Traitement de {filing_url}")
        
        headers = {"User-Agent": SEC_USER_AGENT}
        response = requests.get(filing_url, headers=headers, timeout=30)
        response.raise_for_status()
        
        # --- NETTOYAGE DRASTIQUE ---
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # On supprime les styles, scripts et SURTOUT les tables (qui pèsent lourd)
        for tag in soup(["script", "style", "table", "footer", "header"]):
            tag.decompose()
        
        # Extraction du texte narratif uniquement
        clean_text = soup.get_text(separator=' ', strip=True)
        
        # --- CRÉATION DU DOCUMENT UNIQUE ---
        # On ne fait plus de boucle, on crée un seul objet Document
        doc = Document(
            page_content=clean_text,
            metadata={
                "source": "sec_edgar_full",
                "company": filing_metadata.get("company"),
                "cik": filing_metadata.get("cik"),
                "date": filing_metadata.get("filing_date"),
                "url": filing_url,
                "type": filing_metadata.get("filing_type"),
                "namespace": "financial_reports" # Pour ton index Pinecone
            }
        )
        
        log_ingestion("SEC_EDGAR_FULL", "SUCCESS", f"Rapport compressé créé pour {filing_metadata.get('company')}")
        return [doc]
        
    except Exception as e:
        log_ingestion("SEC_EDGAR_FULL", "ERROR", f"Erreur sur {filing_url}: {str(e)}")
        return []

# ═══════════════════════════════════════════════════════════════
# SOURCE 2 : NEWSAPI 
# ═══════════════════════════════════════════════════════════════

def load_newsapi_articles(query: str, language: str = "en", days_back: int = 7) -> List[Document]:
    """Charge des articles depuis NewsAPI"""
    if not NEWSAPI_KEY:
        log_ingestion("NEWSAPI", "ERROR", "Clé API manquante")
        return []
    
    try:
        from_date = (datetime.now() - timedelta(days=days_back)).strftime("%Y-%m-%d")
        
        params = {
            "q": query,
            "from": from_date,
            "language": language,
            "sortBy": "relevancy",
            "pageSize": 100,  # Max 100
            "apiKey": NEWSAPI_KEY
        }
        
        log_ingestion("NEWSAPI", "INFO", f"Recherche : '{query}'")
        
        response = requests.get(NEWSAPI_ENDPOINT, params=params, timeout=10)
        response.raise_for_status()
        
        data = response.json()
        articles = data.get("articles", [])
        
        documents = []
        for article in articles:
            content = f"{article.get('title', '')}. {article.get('description', '')}"
            
            doc = Document(
                page_content=content,
                metadata={
                    "source": "newsapi",
                    "title": article.get("title"),
                    "author": article.get("author") or "Unknown",  # Fix None
                    "published_at": article.get("publishedAt"),
                    "url": article.get("url"),
                    "source_name": article.get("source", {}).get("name"),
                    "namespace": "news"
                }
            )
            documents.append(doc)
        
        log_ingestion("NEWSAPI", "SUCCESS", f"{len(documents)} articles récupérés")
        return documents
    
    except Exception as e:
        log_ingestion("NEWSAPI", "ERROR", str(e))
        return []

# ═══════════════════════════════════════════════════════════════
# SOURCE 3 : COMMUNIQUÉS DE PRESSE 
# ═══════════════════════════════════════════════════════════════

def load_press_releases(urls: List[str], chunk_articles: bool = True) -> List[Document]:
    """
    Charge des communiqués de presse avec chunking automatique.
    
    AMÉLIORATIONS par rapport à la version précédente :
    - Chunking automatique des articles longs
    - Nettoyage HTML (suppression menus, footers)
    - Support des URLs multiples en masse
    
    Paramètres :
    -----------
    urls : List[str]
        Liste des URLs de communiqués individuels (pas les pages d'accueil)
    chunk_articles : bool
        Si True, découpe les articles longs en chunks (recommandé pour RAG)
    
    Exemple d'utilisation :
    ----------------------
    # Scraper 100 articles individuels au lieu d'1 page d'accueil
    press_urls = [
        "https://www.apple.com/newsroom/2024/12/apple-announces-q4-results/",
        "https://www.apple.com/newsroom/2024/11/apple-launches-new-product/",
        # ... 98 autres URLs
    ]
    docs = load_press_releases(press_urls, chunk_articles=True)
    # Résultat : 100-200 documents au lieu de 1
    """
    
    documents = []
    
    for url in urls:
        try:
            log_ingestion("PRESS_RELEASE", "INFO", f"Scraping {url}")
            
            # ─────────────────────────────────────────────────────
            # 1. CHARGEMENT DE LA PAGE
            # ─────────────────────────────────────────────────────
            
            loader = WebBaseLoader(url)
            docs = loader.load()
            
            if not docs:
                log_ingestion("PRESS_RELEASE", "WARNING", f"Aucun contenu trouvé pour {url}")
                continue
            
            # ─────────────────────────────────────────────────────
            # 2. NETTOYAGE DU TEXTE (supprimer le bruit HTML)
            # ─────────────────────────────────────────────────────
            
            raw_doc = docs[0]
            raw_text = raw_doc.page_content
            
            # Parsing avec BeautifulSoup pour un nettoyage plus précis
            soup = BeautifulSoup(raw_text, 'html.parser')
            
            # Supprimer navigation, footer, scripts
            for tag in soup(["nav", "footer", "script", "style", "aside"]):
                tag.decompose()
            
            # Extraire le texte propre
            clean_text = soup.get_text(separator='\n', strip=True)
            
            # Supprimer lignes vides multiples
            lines = [line for line in clean_text.split('\n') if line.strip()]
            clean_text = '\n'.join(lines)
            
            # ─────────────────────────────────────────────────────
            # 3. CHUNKING (si l'article est long)
            # ─────────────────────────────────────────────────────
            
            if chunk_articles and len(clean_text) > 2000:  # Articles > 2000 caractères
                from langchain_text_splitters import RecursiveCharacterTextSplitter
                
                splitter = RecursiveCharacterTextSplitter(
                    chunk_size=1000,
                    chunk_overlap=150,
                    separators=["\n\n", "\n", ". ", " ", ""]
                )
                
                chunks = splitter.split_text(clean_text)
                
                # Créer un document par chunk
                for i, chunk in enumerate(chunks):
                    doc = Document(
                        page_content=chunk,
                        metadata={
                            "source": "press_release",
                            "url": url,
                            "scrape_date": datetime.now().isoformat(),
                            "chunk_index": i,
                            "total_chunks": len(chunks),
                            "namespace": "news"
                        }
                    )
                    documents.append(doc)
                
                log_ingestion("PRESS_RELEASE", "SUCCESS", 
                    f"Article chargé et chunké en {len(chunks)} parties depuis {url}")
            
            else:
                # Article court, pas de chunking
                doc = Document(
                    page_content=clean_text,
                    metadata={
                        "source": "press_release",
                        "url": url,
                        "scrape_date": datetime.now().isoformat(),
                        "namespace": "news"
                    }
                )
                documents.append(doc)
                log_ingestion("PRESS_RELEASE", "SUCCESS", f"Article court chargé depuis {url}")
            
            time.sleep(2)  # Rate limiting éthique
        
        except Exception as e:
            log_ingestion("PRESS_RELEASE", "ERROR", f"{url} - {str(e)}")
            continue
    
    return documents

# ═══════════════════════════════════════════════════════════════
# SOURCE 4 : GOOGLE NEWS RSS
# ═══════════════════════════════════════════════════════════════

def load_google_news_rss(query: str, limit: int = 10) -> List[Document]:
    """
    Charge des actualités depuis Google News RSS (GRATUIT)
    
    Alternative à Google News API qui n'existe plus en version gratuite.
    Utilise les flux RSS publics de Google News.
    """
    try:
        # URL du flux RSS Google News
        # Format : https://news.google.com/rss/search?q=QUERY&hl=en&gl=US
        query_encoded = requests.utils.quote(query)
        rss_url = f"https://news.google.com/rss/search?q={query_encoded}&hl=en&gl=US&ceid=US:en"
        
        log_ingestion("GOOGLE_NEWS_RSS", "INFO", f"Flux RSS pour '{query}'")
        
        # Parser le flux RSS avec feedparser
        feed = feedparser.parse(rss_url)
        
        documents = []
        for entry in feed.entries[:limit]:
            content = f"{entry.title}. {entry.get('summary', '')}"
            
            doc = Document(
                page_content=content,
                metadata={
                    "source": "google_news_rss",
                    "title": entry.title,
                    "published_at": entry.get("published", "Unknown"),
                    "url": entry.link,
                    "source_name": entry.get("source", {}).get("title", "Google News"),
                    "namespace": "news"
                }
            )
            documents.append(doc)
        
        log_ingestion("GOOGLE_NEWS_RSS", "SUCCESS", f"{len(documents)} articles récupérés")
        return documents
    
    except Exception as e:
        log_ingestion("GOOGLE_NEWS_RSS", "ERROR", str(e))
        return []

# ═══════════════════════════════════════════════════════════════
# SOURCE 5 : YFINANCE --> gérer les problèmes API.
# ═══════════════════════════════════════════════════════════════

def load_yfinance_data(ticker: str, max_retries: int = 3) -> List[Document]:
    """
    Charge les données financières avec gestion du rate limiting
    
    CORRECTIONS :
    - Ajout de délais entre requêtes (6 secondes)
    - Retry en cas d'erreur 429
    - Fallback vers données basiques si quoteSummary échoue
    - User-Agent personnalisé
    """
    try:
        log_ingestion("YFINANCE", "INFO", f"Récupération données pour {ticker}")
        
        # Délai AVANT la requête pour éviter 429
        time.sleep(6)
        
        stock = yf.Ticker(ticker)
        
        # Tentative 1 : Données complètes
        for attempt in range(max_retries):
            try:
                info = stock.info
                
                # Vérifier si on a bien récupéré des données
                if info and 'symbol' in info:
                    break
                
                log_ingestion("YFINANCE", "WARNING", f"Tentative {attempt + 1}/{max_retries} - Données incomplètes")
                time.sleep(10)  # Attente plus longue entre retry
                
            except Exception as e:
                if "429" in str(e):
                    log_ingestion("YFINANCE", "WARNING", f"Rate limit - Retry dans 15s (tentative {attempt + 1})")
                    time.sleep(15)
                else:
                    raise
        
        # Si toujours pas de données, fallback vers historique
        if not info or 'symbol' not in info:
            log_ingestion("YFINANCE", "WARNING", f"Fallback vers données historiques pour {ticker}")
            
            hist = stock.history(period="1d")
            if hist.empty:
                raise ValueError("Aucune donnée disponible")
            
            current_price = hist['Close'].iloc[-1]
            
            content = f"""
            Financial Data for {ticker}:
            - Current Price: ${current_price:.2f}
            - Data Source: Historical prices (quoteSummary unavailable due to rate limiting)
            - Last Updated: {datetime.now().strftime('%Y-%m-%d')}
            """
            
            doc = Document(
                page_content=content,
                metadata={
                    "source": "yfinance",
                    "ticker": ticker,
                    "current_price": float(current_price),
                    "retrieval_date": datetime.now().isoformat(),
                    "namespace": "macro_data",
                    "data_type": "historical_fallback"
                }
            )
            
            log_ingestion("YFINANCE", "SUCCESS", f"Données historiques récupérées pour {ticker}")
            return [doc]
        
        # Données complètes disponibles
        metrics = {
            "market_cap": info.get("marketCap"),
            "revenue": info.get("totalRevenue"),
            "profit_margin": info.get("profitMargins"),
            "pe_ratio": info.get("trailingPE"),
            "current_price": info.get("currentPrice"),
            "52week_high": info.get("fiftyTwoWeekHigh"),
            "52week_low": info.get("fiftyTwoWeekLow")
        }
        
        # Formatage sécurisé
        def format_value(v, prefix="$", suffix="", format_spec=","):
            if v is None:
                return "N/A"
            if isinstance(v, (int, float)):
                return f"{prefix}{v:{format_spec}}{suffix}"
            return str(v)
        
        content = f"""
        Financial Overview for {ticker}:
        - Market Cap: {format_value(metrics['market_cap'])}
        - Revenue: {format_value(metrics['revenue'])}
        - Profit Margin: {format_value(metrics['profit_margin'], prefix="", suffix="%", format_spec=".2f")}
        - P/E Ratio: {format_value(metrics['pe_ratio'], prefix="", format_spec=".2f")}
        - Current Price: {format_value(metrics['current_price'], format_spec=".2f")}
        - 52-Week Range: {format_value(metrics['52week_low'], format_spec=".2f")} - {format_value(metrics['52week_high'], format_spec=".2f")}
        """
        
        doc = Document(
            page_content=content,
            metadata={
                "source": "yfinance",
                "ticker": ticker,
                "company_name": info.get("longName", ticker),
                "sector": info.get("sector") or "Unknown",
                "industry": info.get("industry") or "Unknown",
                "retrieval_date": datetime.now().isoformat(),
                "namespace": "macro_data",
                **{k: v for k, v in metrics.items() if v is not None}
            }
        )
        
        log_ingestion("YFINANCE", "SUCCESS", f"Données complètes récupérées pour {ticker}")
        return [doc]
    
    except Exception as e:
        log_ingestion("YFINANCE", "ERROR", f"{ticker} - {str(e)}")
        return []


# ═══════════════════════════════════════════════════════════════
# SOURCE 6 : FRED 
# ═══════════════════════════════════════════════════════════════

def load_fred_macro_data(series_ids: List[str] = ["GDP", "CPIAUCSL", "FEDFUNDS", "UNRATE"]) -> List[Document]:
    """
    Charge les indicateurs macro-économiques depuis St. Louis FRED.
    Par défaut : PIB, Inflation (CPI), Taux de la FED, Chômage.
    """
    if not FRED_API_KEY:
        log_ingestion("FRED", "ERROR", "Clé API FRED_API_KEY manquante dans .env")
        return []

    documents = []
    
    for series_id in series_ids:
        try:
            log_ingestion("FRED", "INFO", f"Récupération de la série : {series_id}")
            
            # 1. Récupérer les infos de la série (nom, unités)
            info_url = f"{FRED_BASE_URL}series?series_id={series_id}&api_key={FRED_API_KEY}&file_type=json"
            info_resp = requests.get(info_url, timeout=10)
            info_resp.raise_for_status()
            series_info = info_resp.json()['seriess'][0]
            
            # 2. Récupérer les dernières observations (les 5 dernières pour le contexte)
            obs_url = f"{FRED_BASE_URL}series/observations?series_id={series_id}&api_key={FRED_API_KEY}&sort_order=desc&limit=5&file_type=json"
            obs_resp = requests.get(obs_url, timeout=10)
            obs_resp.raise_for_status()
            observations = obs_resp.json()['observations']

            # 3. Création du contenu textuel pour le RAG
            title = series_info.get('title', series_id)
            units = series_info.get('units_short', '')
            obs_text = "\n".join([f"- Date: {o['date']}, Valeur: {o['value']} {units}" for o in observations])
            
            content = f"Série Économique : {title} ({series_id}).\n" \
                      f"Description : {series_info.get('notes', 'N/A')}\n" \
                      f"Dernières observations :\n{obs_text}"

            # 4. Formatage Document (compatible avec votre pipeline)
            doc = Document(
                page_content=content,
                metadata={
                    "source": "fred",
                    "series_id": series_id,
                    "title": title,
                    "last_updated": observations[0]['date'] if observations else "Unknown",
                    "namespace": "macro_data"  # Envoi direct dans le bon namespace Pinecone
                }
            )
            documents.append(doc)
            
            time.sleep(0.5) # Respect du rate limit de la FRED
            log_ingestion("FRED", "SUCCESS", f"Série {series_id} chargée avec succès")

        except Exception as e:
            log_ingestion("FRED", "ERROR", f"Erreur pour {series_id}: {str(e)}")
            continue

    return documents


# ═══════════════════════════════════════════════════════════════
# SOURCE : API SIRENE (INSEE - FRANCE)
# ═══════════════════════════════════════════════════════════════

def load_sirene_data_by_siren(siren_list, delay_seconds=0.5):

    headers = {
        "X-INSEE-Api-Key-Integration": API_KEY,
        "Accept": "application/json",
        "User-Agent": "curl/8.4.0"
    }

    documents = []

    for siren in siren_list:
        siren = siren.strip()
        url = f"{BASE_URL}/siren/{siren}"

        log_ingestion("SIRENE", "INFO", f"Requête SIREN {siren}")

        try:
            # ─────────────────────────────────────────────
            # APPEL API (curl-like)
            # ─────────────────────────────────────────────
            response = requests.get(url, headers=headers, timeout=15)

            if response.status_code != 200:
                log_ingestion("SIRENE", "ERROR", f"SIREN {siren} - HTTP {response.status_code}")
                continue

            data = response.json()
            unite = data.get("uniteLegale", {})

            if not unite:
                log_ingestion("SIRENE", "ERROR", f"SIREN {siren} - JSON vide")
                continue

            # ─────────────────────────────────────────────
            # TRI DU JSON (ON GARDE L’UTILE)
            # ─────────────────────────────────────────────

            # période courante = dateFin == null
            current_period = None
            for p in unite.get("periodesUniteLegale", []):
                if p.get("dateFin") is None:
                    current_period = p
                    break

            if not current_period:
                log_ingestion("SIRENE", "ERROR", f"SIREN {siren} - période courante absente")
                continue

            # anciens noms
            old_names = []
            old_activities = []

            for p in unite.get("periodesUniteLegale", []):
                name = p.get("denominationUniteLegale")
                activity = p.get("activitePrincipaleUniteLegale")

                if name and name not in old_names:
                    old_names.append(name)

                if activity and activity not in old_activities:
                    old_activities.append(activity)

            # JSON nettoyé (MENTALEMENT SIMPLE)
            cleaned = {
                "siren": unite.get("siren"),
                "date_creation": unite.get("dateCreationUniteLegale"),
                "categorie_entreprise": unite.get("categorieEntreprise"),
                "effectifs": {
                    "tranche": unite.get("trancheEffectifsUniteLegale"),
                    "annee": unite.get("anneeEffectifsUniteLegale")
                },
                "current": {
                    "denomination": current_period.get("denominationUniteLegale"),
                    "naf": current_period.get("activitePrincipaleUniteLegale"),
                    "categorie_juridique": current_period.get("categorieJuridiqueUniteLegale"),
                    "etat": current_period.get("etatAdministratifUniteLegale")
                },
                "history": {
                    "denominations": old_names,
                    "activities": old_activities
                }
            }

            # ─────────────────────────────────────────────
            # TRANSFORMATION POUR LE PIPELINE (Document)
            # ─────────────────────────────────────────────

            page_content = f"""
            ENTREPRISE – INSEE SIRENE
            SIREN : {cleaned['siren']}
            Dénomination actuelle : {cleaned['current']['denomination']}
            Date de création : {cleaned['date_creation']}
            Catégorie entreprise : {cleaned['categorie_entreprise']}
            Effectifs : {cleaned['effectifs']['tranche']} (année {cleaned['effectifs']['annee']})
            Activité principale (NAF) : {cleaned['current']['naf']}
            Catégorie juridique : {cleaned['current']['categorie_juridique']}
            État administratif : {cleaned['current']['etat']}

            Historique des noms :
            {' → '.join(cleaned['history']['denominations'])}

            Historique des activités :
            {', '.join(cleaned['history']['activities'])}
            """.strip()

            documents.append(
                Document(
                    page_content=page_content,
                    metadata={
                        "source": "INSEE_SIRENE",
                        "siren": cleaned["siren"],
                        "country": "FR"
                    }
                )
            )

            log_ingestion("SIRENE", "SUCCESS", f"SIREN {siren} ingéré")

        except Exception as e:
            log_ingestion("SIRENE", "ERROR", f"SIREN {siren} - {str(e)}")

        time.sleep(delay_seconds)

    return documents

# ═══════════════════════════════════════════════════════════════
# SOURCE : DBNOMICS (DONNÉES MACRO MONDIALES)
# ═══════════════════════════════════════════════════════════════

def load_dbnomics_data(
    series_list: List[Dict[str, str]],
    max_observations: int = 12
) -> List[Document]:
    """
    Charge des séries économiques depuis DBnomics.
    
    Paramètres :
    -----------
    series_list : List[Dict[str, str]]
        Liste de dictionnaires avec les clés:
        - 'provider': Code du fournisseur (ex: 'OECD')
        - 'dataset': Code du dataset (ex: 'QNA')
        - 'series': Code de la série (ex: 'FRA.B1_GE.CARSA.Q')
        - 'name': (optionnel) Nom descriptif pour le logging
    
    max_observations : int
        Nombre max d'observations à inclure (dernières valeurs)
    
    Exemple d'utilisation :
    ----------------------
    series = [
        {
            'provider': 'OECD',
            'dataset': 'QNA',
            'series': 'FRA.B1_GE.CARSA.Q',
            'name': 'PIB France (trimestriel)'
        },
        {
            'provider': 'OECD',
            'dataset': 'MEI',
            'series': 'FRA.LR.STSA.M',
            'name': 'Taux chômage France'
        }
    ]
    
    docs = load_dbnomics_data(series)
    """
    
    try:
        documents = []
        
        log_ingestion("DBNOMICS", "INFO", f"Récupération de {len(series_list)} séries")
        
        # ─────────────────────────────────────────────────────────
        # RÉCUPÉRATION DE CHAQUE SÉRIE
        # ─────────────────────────────────────────────────────────
        
        for series_info in series_list:
            provider = series_info['provider']
            dataset = series_info['dataset']
            series_code = series_info['series']
            series_name = series_info.get('name', f"{provider}/{dataset}/{series_code}")
            
            try:
                log_ingestion("DBNOMICS", "INFO", f"Fetch: {series_name}")
                
                # Appel API DBnomics
                df = db.fetch_series(provider, dataset, series_code)
                
                if df is None or df.empty:
                    log_ingestion("DBNOMICS", "WARNING", f"Série vide: {series_name}")
                    continue
                
                # Formatage en Document LangChain
                doc = _format_dbnomics_series(df, max_observations, series_name)
                documents.append(doc)
                
                # Rate limiting éthique
                time.sleep(0.5)
            
            except Exception as e:
                log_ingestion("DBNOMICS", "ERROR", f"{series_name}: {str(e)}")
                continue
        
        # ─────────────────────────────────────────────────────────
        # LOGGING FINAL
        # ─────────────────────────────────────────────────────────
        
        log_ingestion("DBNOMICS", "SUCCESS", f"{len(documents)} séries récupérées")
        return documents
    
    except Exception as e:
        log_ingestion("DBNOMICS", "ERROR", f"Erreur globale: {str(e)}")
        return []


def _format_dbnomics_series(df: pd.DataFrame, max_obs: int, series_name: str) -> Document:
    """
    Formate une série DBnomics en Document LangChain.
    
    Fonction utilitaire interne.
    """
    
    # Tri par date décroissante
    df_sorted = df.sort_values('period', ascending=False).head(max_obs)
    
    # Extraction des métadonnées (colonnes toujours présentes)
    provider = df['provider_code'].iloc[0]
    dataset = df['dataset_code'].iloc[0]
    full_series_code = df['series_code'].iloc[0]
    
    # Informations optionnelles (peuvent ne pas exister)
    frequency = df.get('@frequency', pd.Series(['Unknown'] * len(df))).iloc[0]
    unit = df.get('unit', pd.Series([''] * len(df))).iloc[0]
    
    # Construction du texte des observations
    observations_text = []
    for _, row in df_sorted.iterrows():
        period = row['period']
        value = row['value']
        
        if pd.notna(value):
            if isinstance(value, (int, float)):
                value_str = f"{value:,.2f}"
            else:
                value_str = str(value)
        else:
            value_str = "N/A"
        
        observations_text.append(f"  - {period} : {value_str} {unit}")
    
    # Contenu pour le RAG
    content = f"""
    SÉRIE ÉCONOMIQUE DBnomics

    Source : {provider} / {dataset}
    Série : {series_name}
    Code complet : {full_series_code}
    Fréquence : {frequency}
    Unité : {unit}

    Dernières observations ({len(df_sorted)} valeurs) :
    {chr(10).join(observations_text)}

    Dernière mise à jour : {df_sorted['period'].iloc[0]}
        """.strip()
    
    # Calcul de statistiques
    values = df_sorted['value'].dropna()
    
    metadata = {
        "source": "dbnomics",
        "provider": provider,
        "dataset": dataset,
        "series_code": full_series_code,
        "series_name": series_name,
        "frequency": str(frequency),
        "unit": str(unit),
        "last_period": str(df_sorted['period'].iloc[0]),
        "observations_count": len(df_sorted),
        "namespace": "macro_data"
    }
    
    # Ajout des stats si données numériques
    if not values.empty and len(values) > 1:
        metadata["last_value"] = float(values.iloc[0])
        metadata["mean"] = float(values.mean())
        metadata["min"] = float(values.min())
        metadata["max"] = float(values.max())
    
    return Document(
        page_content=content,
        metadata=metadata
    )

# ═══════════════════════════════════════════════════════════════
# SOURCE : ADZUNA (OFFRES D'EMPLOI & MARCHÉ DU TRAVAIL)
# ═══════════════════════════════════════════════════════════════

def load_adzuna_data(
    what: str = None,
    where: str = None,
    country: str = "fr",
    results_per_page: int = 20,
    page: int = 1,
    salary_min: int = None,
    category: str = None,
    endpoint_type: str = "search"
) -> List[Document]:
    """
    Charge des données du marché de l'emploi depuis l'API Adzuna.
    
    Paramètres :
    -----------
    what : str
        Mots-clés de recherche (ex: "data scientist", "consultant stratégie")
    where : str
        Localisation (ex: "Paris", "Lyon", "France")
    country : str
        Code pays (fr, gb, us, de, etc.)
    results_per_page : int
        Nombre de résultats par page (max 50)
    page : int
        Numéro de page (pagination)
    salary_min : int
        Salaire minimum annuel
    category : str
        Catégorie d'emploi (ex: "it-jobs", "consultancy-jobs")
    endpoint_type : str
        Type de données : "search", "histogram", "top_companies"
    
    Exemples d'utilisation :
    ------------------------
    # Recherche d'offres
    load_adzuna_data(what="data scientist", where="Paris", results_per_page=30)
    
    # Tendances salariales
    load_adzuna_data(what="consultant", endpoint_type="histogram")
    
    # Top recruteurs
    load_adzuna_data(what="fintech", endpoint_type="top_companies")
    """
    
    APP_ID = os.getenv("ADZUNA_APP_ID")
    APP_KEY = os.getenv("ADZUNA_APP_KEY")
    
    if not APP_ID or not APP_KEY:
        log_ingestion("ADZUNA", "ERROR", "Clés ADZUNA_APP_ID ou ADZUNA_APP_KEY manquantes dans .env")
        return []
    
    try:
        documents = []
        
        # ─────────────────────────────────────────────────────────
        # 1. CONSTRUCTION DE L'URL SELON LE TYPE D'ENDPOINT
        # ─────────────────────────────────────────────────────────
        
        BASE_URL = "https://api.adzuna.com/v1/api/jobs"
        
        if endpoint_type == "search":
            url = f"{BASE_URL}/{country}/search/{page}"
        elif endpoint_type == "histogram":
            url = f"{BASE_URL}/{country}/histogram"
        elif endpoint_type == "top_companies":
            url = f"{BASE_URL}/{country}/top_companies"
        else:
            log_ingestion("ADZUNA", "ERROR", f"Type d'endpoint inconnu: {endpoint_type}")
            return []
        
        # ─────────────────────────────────────────────────────────
        # 2. PARAMÈTRES DE LA REQUÊTE
        # ─────────────────────────────────────────────────────────
        
        params = {
            "app_id": APP_ID,
            "app_key": APP_KEY,
            "content-type": "application/json"
        }
        
        # Ajout des paramètres optionnels
        if what:
            params["what"] = what
        if where:
            params["where"] = where
        if results_per_page and endpoint_type == "search":
            params["results_per_page"] = min(results_per_page, 50)  # Max 50 par page
        if salary_min:
            params["salary_min"] = salary_min
        if category:
            params["category"] = category
        
        log_ingestion("ADZUNA", "INFO", 
            f"Requête {endpoint_type}: what='{what}', where='{where}', country={country}")
        
        # ─────────────────────────────────────────────────────────
        # 3. APPEL API
        # ─────────────────────────────────────────────────────────
        
        response = requests.get(url, params=params, timeout=15)
        
        # Debug
        if response.status_code != 200:
            log_ingestion("ADZUNA", "ERROR", 
                f"Code {response.status_code} - URL: {response.url[:100]}...")
        
        # Gestion des erreurs
        if response.status_code == 401:
            log_ingestion("ADZUNA", "ERROR", "Authentification échouée - Vérifiez vos clés API")
            return []
        elif response.status_code == 429:
            log_ingestion("ADZUNA", "WARNING", "Rate limit atteint - Patientez 1 minute")
            return []
        
        response.raise_for_status()
        data = response.json()
        
        # ─────────────────────────────────────────────────────────
        # 4. PARSING SELON LE TYPE D'ENDPOINT
        # ─────────────────────────────────────────────────────────
        
        if endpoint_type == "search":
            documents = _parse_adzuna_jobs(data, what, where, country)
        
        elif endpoint_type == "histogram":
            documents = _parse_adzuna_histogram(data, what, where, country)
        
        elif endpoint_type == "top_companies":
            documents = _parse_adzuna_top_companies(data, what, where, country)
        
        # ─────────────────────────────────────────────────────────
        # 5. LOGGING FINAL
        # ─────────────────────────────────────────────────────────
        
        log_ingestion("ADZUNA", "SUCCESS", f"{len(documents)} documents créés")
        return documents
    
    except requests.exceptions.RequestException as e:
        log_ingestion("ADZUNA", "ERROR", f"Erreur réseau: {str(e)}")
        return []
    except Exception as e:
        log_ingestion("ADZUNA", "ERROR", f"Erreur: {str(e)}")
        return []


# ═══════════════════════════════════════════════════════════════
# FONCTIONS UTILITAIRES DE PARSING
# ═══════════════════════════════════════════════════════════════

def _parse_adzuna_jobs(data: dict, what: str, where: str, country: str) -> List[Document]:
    """Parse les offres d'emploi"""
    
    documents = []
    results = data.get("results", [])
    count = data.get("count", 0)
    mean_salary = data.get("mean", 0)
    
    log_ingestion("ADZUNA", "INFO", 
        f"Trouvé {count} offres (moyenne salaire: {mean_salary:.0f}€)")
    
    for job in results:
        title = job.get("title", "N/A")
        company = job.get("company", {}).get("display_name", "N/A")
        location = job.get("location", {}).get("display_name", "N/A")
        description = job.get("description", "")[:500]  # Limiter la description
        
        salary_min = job.get("salary_min")
        salary_max = job.get("salary_max")
        salary_text = ""
        if salary_min and salary_max:
            salary_text = f"{salary_min:,.0f}€ - {salary_max:,.0f}€"
        elif salary_min:
            salary_text = f"À partir de {salary_min:,.0f}€"
        else:
            salary_text = "Non communiqué"
        
        created = job.get("created", "N/A")
        category = job.get("category", {}).get("label", "N/A")
        contract_type = job.get("contract_type", "N/A")
        
        # Contenu pour le RAG
        content = f"""
OFFRE D'EMPLOI - Adzuna

Poste : {title}
Entreprise : {company}
Localisation : {location}

Rémunération : {salary_text}
Type de contrat : {contract_type}
Catégorie : {category}
Date de publication : {created}

Description :
{description}...
        """.strip()
        
        # Métadonnées
        metadata = {
            "source": "adzuna",
            "job_title": title,
            "company": company,
            "location": location,
            "salary_min": salary_min,
            "salary_max": salary_max,
            "category": category,
            "contract_type": contract_type,
            "created": created,
            "search_query": what or "",
            "search_location": where or "",
            "country": country,
            "namespace": "macro_data",  # Ou "news" selon votre logique
            "url": job.get("redirect_url", "")
        }
        
        documents.append(Document(page_content=content, metadata=metadata))
    
    return documents


def _parse_adzuna_histogram(data: dict, what: str, where: str, country: str) -> List[Document]:
    """Parse la distribution salariale (histogram)"""
    
    histogram = data.get("histogram", {})
    
    if not histogram:
        log_ingestion("ADZUNA", "WARNING", "Histogram vide")
        return []
    
    # Formatage des données de distribution
    salary_ranges = []
    for salary_str, count in histogram.items():
        salary_ranges.append(f"  - {salary_str}€ : {count} offres")
    
    content = f"""
ANALYSE SALARIALE - Adzuna

Recherche : {what or 'Tous métiers'}
Localisation : {where or 'France entière'}
Pays : {country.upper()}

Distribution des salaires :
{chr(10).join(sorted(salary_ranges))}

Date de l'analyse : {datetime.now().strftime('%Y-%m-%d')}
    """.strip()
    
    metadata = {
        "source": "adzuna",
        "data_type": "salary_histogram",
        "search_query": what or "",
        "search_location": where or "",
        "country": country,
        "namespace": "macro_data",
        "histogram_data": histogram
    }
    
    return [Document(page_content=content, metadata=metadata)]


def _parse_adzuna_top_companies(data: dict, what: str, where: str, country: str) -> List[Document]:
    """Parse le top des entreprises qui recrutent"""
    
    leaderboard = data.get("leaderboard", [])
    
    if not leaderboard:
        log_ingestion("ADZUNA", "WARNING", "Top companies vide")
        return []
    
    # Formatage du classement
    top_companies_text = []
    for i, company_data in enumerate(leaderboard[:20], 1):  # Top 20
        company_name = company_data.get("canonical_name", "N/A")
        count = company_data.get("count", 0)
        top_companies_text.append(f"  {i}. {company_name} : {count} offres")
    
    content = f"""
TOP RECRUTEURS - Adzuna

Recherche : {what or 'Tous secteurs'}
Localisation : {where or 'France entière'}

Classement des entreprises qui recrutent le plus :
{chr(10).join(top_companies_text)}

Date de l'analyse : {datetime.now().strftime('%Y-%m-%d')}
Total entreprises analysées : {len(leaderboard)}
    """.strip()
    
    metadata = {
        "source": "adzuna",
        "data_type": "top_companies",
        "search_query": what or "",
        "search_location": where or "",
        "country": country,
        "namespace": "macro_data",
        "top_20": [c.get("canonical_name") for c in leaderboard[:20]]
    }
    
    return [Document(page_content=content, metadata=metadata)]


# ═══════════════════════════════════════════════════════════════
# SOURCE : BLUESKY (SIGNAUX SOCIAUX & VEILLE)
# ═══════════════════════════════════════════════════════════════

def load_bluesky_data(
    search_queries: List[str] = None,
    author_handle: str = None,
    limit: int = 25,
    lang: str = "fr",
    since_days: int = 7
) -> List[Document]:
    """
    Charge des posts depuis Bluesky pour la veille stratégique.
    
    Paramètres :
    -----------
    search_queries : List[str]
        Liste de requêtes de recherche (ex: ["fintech", "LVMH", "data science"])
    author_handle : str
        Handle d'un auteur spécifique (ex: "company.bsky.social")
    limit : int
        Nombre max de posts par requête (max 100)
    lang : str
        Code langue (fr, en, etc.)
    since_days : int
        Chercher les posts des X derniers jours
    
    Exemples d'utilisation :
    ------------------------
    # Recherche par mots-clés
    load_bluesky_data(search_queries=["fintech France", "AI startup"], limit=30)
    
    # Posts d'un compte spécifique
    load_bluesky_data(author_handle="techcrunch.bsky.social", limit=20)
    
    # Veille multi-thèmes
    load_bluesky_data(search_queries=["KPMG", "audit digital", "ESG reporting"])
    """
    
    HANDLE = os.getenv("BLUESKY_HANDLE")
    PASSWORD = os.getenv("BLUESKY_APP_PASSWORD")
    
    if not HANDLE or not PASSWORD:
        log_ingestion("BLUESKY", "ERROR", 
            "Credentials BLUESKY_HANDLE ou BLUESKY_APP_PASSWORD manquants")
        return []
    
    try:
        documents = []
        
        # ─────────────────────────────────────────────────────────
        # 1. AUTHENTIFICATION
        # ─────────────────────────────────────────────────────────
        
        log_ingestion("BLUESKY", "INFO", "Connexion à Bluesky...")
        
        client = Client()
        profile = client.login(HANDLE, PASSWORD)
        
        log_ingestion("BLUESKY", "SUCCESS", f"Connecté en tant que {profile.display_name}")
        
        # ─────────────────────────────────────────────────────────
        # 2. COLLECTE DES POSTS
        # ─────────────────────────────────────────────────────────
        
        if search_queries:
            # Mode recherche par mots-clés
            for query in search_queries:
                log_ingestion("BLUESKY", "INFO", f"Recherche: '{query}'")
                
                try:
                    # Appel à l'API de recherche
                    response = client.app.bsky.feed.search_posts(
                        params={
                            'q': query,
                            'limit': limit,
                            'lang': lang
                        }
                    )
                    
                    posts = response.posts if hasattr(response, 'posts') else []
                    
                    log_ingestion("BLUESKY", "INFO", 
                        f"Trouvé {len(posts)} posts pour '{query}'")
                    
                    # Parsing des posts
                    for post in posts:
                        doc = _parse_bluesky_post(post, query)
                        if doc:
                            documents.append(doc)
                    
                    time.sleep(1)  # Rate limiting
                
                except Exception as e:
                    log_ingestion("BLUESKY", "ERROR", 
                        f"Erreur recherche '{query}': {str(e)}")
                    continue
        
        elif author_handle:
            # Mode posts d'un auteur spécifique
            log_ingestion("BLUESKY", "INFO", f"Posts de @{author_handle}")
            
            try:
                response = client.app.bsky.feed.get_author_feed(
                    params={
                        'actor': author_handle,
                        'limit': limit
                    }
                )
                
                feed = response.feed if hasattr(response, 'feed') else []
                
                log_ingestion("BLUESKY", "INFO", 
                    f"Trouvé {len(feed)} posts de @{author_handle}")
                
                for item in feed:
                    post = item.post
                    doc = _parse_bluesky_post(post, f"author:{author_handle}")
                    if doc:
                        documents.append(doc)
            
            except Exception as e:
                log_ingestion("BLUESKY", "ERROR", 
                    f"Erreur récupération feed @{author_handle}: {str(e)}")
        
        else:
            log_ingestion("BLUESKY", "ERROR", 
                "Aucun critère de recherche fourni (search_queries ou author_handle)")
            return []
        
        # ─────────────────────────────────────────────────────────
        # 3. LOGGING FINAL
        # ─────────────────────────────────────────────────────────
        
        log_ingestion("BLUESKY", "SUCCESS", f"{len(documents)} posts collectés")
        return documents
    
    except Exception as e:
        log_ingestion("BLUESKY", "ERROR", f"Erreur globale: {str(e)}")
        return []


# ═══════════════════════════════════════════════════════════════
# FONCTION UTILITAIRE DE PARSING
# ═══════════════════════════════════════════════════════════════

def _parse_bluesky_post(post, search_query: str = "") -> Document:
    """
    Parse un post Bluesky en Document LangChain.
    
    Fonction utilitaire interne.
    """
    
    try:
        # Extraction des données du post
        author = post.author
        record = post.record
        
        author_handle = author.handle
        author_name = author.display_name or author_handle
        
        # Texte du post
        text = record.text
        
        # Date de création
        created_at_str = record.created_at
        created_at = datetime.fromisoformat(created_at_str.replace('Z', '+00:00'))
        
        # Engagement metrics
        like_count = post.like_count or 0
        repost_count = post.repost_count or 0
        reply_count = post.reply_count or 0
        
        # URI du post (pour créer le lien)
        post_uri = post.uri
        # Format URI: at://did:plc:xxx/app.bsky.feed.post/yyy
        # Conversion en URL web
        post_id = post_uri.split('/')[-1]
        post_url = f"https://bsky.app/profile/{author_handle}/post/{post_id}"
        
        # Contenu pour le RAG
        content = f"""
POST BLUESKY - Veille Sociale

Auteur : @{author_handle} ({author_name})
Date : {created_at.strftime('%Y-%m-%d %H:%M')}

Contenu :
{text}

Engagement :
-   {like_count} likes
-  {repost_count} reposts
-  {reply_count} réponses

Lien : {post_url}
Recherche associée : {search_query}
        """.strip()
        
        # Métadonnées
        metadata = {
            "source": "bluesky",
            "author_handle": author_handle,
            "author_name": author_name,
            "created_at": created_at.isoformat(),
            "like_count": like_count,
            "repost_count": repost_count,
            "reply_count": reply_count,
            "engagement_total": like_count + repost_count + reply_count,
            "post_url": post_url,
            "post_uri": post_uri,
            "search_query": search_query,
            "namespace": "news",  # Ou "macro_data" selon votre logique
            "text_length": len(text)
        }
        
        # Ajout de tags si présents
        if hasattr(record, 'tags') and record.tags:
            metadata["tags"] = record.tags
        
        # Détection de langue (si disponible)
        if hasattr(record, 'langs') and record.langs:
            metadata["language"] = record.langs[0]
        
        return Document(
            page_content=content,
            metadata=metadata
        )
    
    except Exception as e:
        log_ingestion("BLUESKY", "WARNING", f"Erreur parsing post: {str(e)}")
        return None
    

# ═══════════════════════════════════════════════════════════════
# PIPELINE D'INGESTION COMPLET
# ═══════════════════════════════════════════════════════════════

    """
    Lance l'ingestion de TOUTES les sources configurées.
    
    Sources incluses :
    - SEC EDGAR (métadonnées + documents complets)
    - NewsAPI (20 queries stratégiques)
    - Google News RSS
    - Communiqués de presse
    - yfinance (Top 50 S&P500)
    - FRED (données macro US)
    - DBnomics (données macro mondiales)
    - Bluesky (signaux sociaux)
    - INSEE SIRENE (entreprises françaises)
    - Adzuna (marché de l'emploi)
    """


def ingest_all_sources() -> Dict[str, List[Document]]:
    """Lance l'ingestion de toutes les sources"""
    
    all_documents = {
        "financial_reports": [],
        "news": [],
        "startups": [],
        "macro_data": [],
        "social_signals": []  # AJOUTER
    }

# "web_quarantine" : Pas besoin (créé dynamiquement)
# "facts" : Pas besoin (créé dans Notebook 3)
    
    print("="*60)
    print(" DÉMARRAGE DE L'INGESTION MULTI-SOURCES")
    print("="*60 + "\n")
    
    # ─────────────────────────────────────────────────────────────
    # 1. SEC EDGAR
    # ─────────────────────────────────────────────────────────────
    print(" Chargement SEC EDGAR...")
    from config import SEC_TARGET_COMPANIES
        # 1A. Chargement des métadonnées (rapide, garde comme référence)
    sec_meta_docs = []
    for cik, company_name in SEC_TARGET_COMPANIES:  
        print(f"    Métadonnées : {company_name}")
        meta_docs = load_sec_edgar_filing(cik, "10-K", limit=1)  # 2 rapports les plus récents
        sec_meta_docs.extend(meta_docs)
        time.sleep(1)  # Rate limiting SEC

    all_documents["financial_reports"].extend(sec_meta_docs)
    print(f"    {len(sec_meta_docs)} métadonnées de rapports chargées.")

    # 1B. Téléchargement des documents complets (NOUVEAU - prend 5-10 min)
    print("    Téléchargement des rapports complets...")
    print("    Patientez ~10 minutes (fichiers lourds)...\n")

    sec_full_docs = []
    for meta_doc in sec_meta_docs:  # Pour chaque métadonnée, charger le document complet
        filing_url = meta_doc.metadata.get("url")
        company = meta_doc.metadata.get("company", "Unknown")
        
        if filing_url:
            print(f"   Téléchargement : {company}")
            chunks = load_sec_full_document(filing_url, meta_doc.metadata)
            sec_full_docs.extend(chunks)
            print(f"      → {len(chunks)} chunks extraits")
            time.sleep(2)  # Rate limiting SEC (important !)

    all_documents["financial_reports"].extend(sec_full_docs)
    print(f"    {len(sec_full_docs)} chunks de documents complets.\n")
    
    # ─────────────────────────────────────────────────────────────
    # 2. NEWSAPI - MULTI-QUERIES STRATÉGIQUES
    # ─────────────────────────────────────────────────────────────
    if NEWSAPI_KEY:
        print("Collecte des Actualités (NewsAPI Multi-Queries)...")
        from config import KPMG_QUERIES_NEWS_API
        
        news_docs = []
        for i, query in enumerate( KPMG_QUERIES_NEWS_API, 1):
            print(f"    Query {i}/{len( KPMG_QUERIES_NEWS_API)}: {query}")
            articles = load_newsapi_articles(query, days_back=30)
            news_docs.extend(articles)
            time.sleep(1)  # Rate limiting
        
        all_documents["news"].extend(news_docs)
        print(f"    {len(news_docs)} articles totaux via NewsAPI.\n")
        
    # ─────────────────────────────────────────────────────────────
    # 3. GOOGLE NEWS RSS (NOUVEAU)
    # ─────────────────────────────────────────────────────────────
    print(" Flux Google News RSS...")
    from config import GOOGLE_RSS_QUERIES
    rss_docs = []
    for i, query in enumerate(GOOGLE_RSS_QUERIES, 1):
        print(f"    RSS Query {i}/{len(GOOGLE_RSS_QUERIES)}: {query}")
        articles = load_google_news_rss(query, limit=25)  # 25 articles par query
        rss_docs.extend(articles)
        time.sleep(random.uniform(1, 2))  # Rate limiting (Google est tolérant mais restons fair)

    all_documents["news"].extend(rss_docs)
    print(f"    {len(rss_docs)} articles totaux via RSS.\n")
    
    # ─────────────────────────────────────────────────────────────
    # 4. COMMUNIQUÉS DE PRESSE
    # ─────────────────────────────────────────────────────────────
    print(" Scraping Communiqués de presse...")
    from config import PRESS_RELEASE_URLS

    press_docs = load_press_releases(PRESS_RELEASE_URLS, chunk_articles=True)
    all_documents["news"].extend(press_docs)
    print(f"    {len(press_docs)} documents de communiqués (articles chunkés).\n")
    
    # ─────────────────────────────────────────────────────────────
    # 5. YFINANCE (AVEC RATE LIMITING)
    # ─────────────────────────────────────────────────────────────

    '''
    print(" Analyse Macro & Marchés (yfinance - Top 50 S&P500)...")
    print("    Patientez ~5 minutes (rate limiting Yahoo Finance)...\n")
    from config import SP500_TOP50_YFINANCE

    finance_docs = []
    for i, ticker in enumerate(SP500_TOP50_YFINANCE, 1):
        print(f"    {i}/50 : {ticker}")
        docs = load_yfinance_data(ticker)
        finance_docs.extend(docs)
        # Respect strict du rate limit (CRITIQUE)
        time.sleep(6)

    all_documents["macro_data"].extend(finance_docs)
    print(f"   {len(finance_docs)} fiches financières récupérées.\n")
    '''
    # ─────────────────────────────────────────────────────────────
    # 7. FRED 
    # ─────────────────────────────────────────────────────────────

    print(" Collecte des indicateurs Macro (FRED)...")
    from config import FRED_SERIES_COMPLETE

    fred_docs = load_fred_macro_data(FRED_SERIES_COMPLETE)
    all_documents["macro_data"].extend(fred_docs)
    print(f"    {len(fred_docs)} séries économiques FRED chargées.\n")

    # ─────────────────────────────────────────────────────────────
    # 8. INSEE SIRENE - ENTREPRISES FRANÇAISES (ÉTENDU)
    # ─────────────────────────────────────────────────────────────
    print(" Collecte des données entreprises françaises (INSEE SIRENE)...")
    from config import SIREN_EXTENDED_INSEE

    sirene_docs = load_sirene_data_by_siren(SIREN_EXTENDED_INSEE, delay_seconds=0.5)
    all_documents["financial_reports"].extend(sirene_docs)
    print(f"    {len(sirene_docs)} entreprises françaises analysées.\n")

    # ─────────────────────────────────────────────────────────────
    # 6. DBNOMICS - INDICATEURS MACRO-ÉCONOMIQUES
    # ─────────────────────────────────────────────────────────────
    # Importer depuis config.py
    from config import KPMG_DBNOMICS_SERIES
        
    dbnomics_docs = load_dbnomics_data(KPMG_DBNOMICS_SERIES, max_observations=10)
    all_documents["macro_data"].extend(dbnomics_docs)
    print(f"   {len(dbnomics_docs)} séries économiques chargées.\n")

    # ─────────────────────────────────────────────────────────────
    # 9. ADZUNA - MARCHÉ DE L'EMPLOI
    # ─────────────────────────────────────────────────────────────
    from config import KPMG_ADZUNA_SEARCHES
        
    adzuna_docs = []
    for search_config in KPMG_ADZUNA_SEARCHES:
        docs = load_adzuna_data(**search_config)
        adzuna_docs.extend(docs)
        time.sleep(1)  # Rate limiting
        
    all_documents["macro_data"].extend(adzuna_docs)
    print(f"    {len(adzuna_docs)} analyses emploi collectées.\n")

    # ─────────────────────────────────────────────────────────────
    # X. BLUESKY (SIGNAUX SOCIAUX)
    # ─────────────────────────────────────────────────────────────
    
    print(" Collecte des signaux sociaux (Bluesky)...")
    from config import ALL_BLUESKY_QUERIES 


    bluesky_docs = load_bluesky_data(
        search_queries=ALL_BLUESKY_QUERIES,
        limit=15,  # 15 posts par requête
        lang="fr"
    )
    
    all_documents["social_signals"].extend(bluesky_docs)
    print(f"   {len(bluesky_docs)} posts Bluesky.\n")
    

    # ─────────────────────────────────────────────────────────────
    # OPTIONNEL : Suivi de comptes spécifiques
    # ─────────────────────────────────────────────────────────────
    
    # Exemple : Suivre les posts d'un média tech
    # influential_accounts = ["techcrunch.bsky.social", "wired.bsky.social"]
    # 
    # for account in influential_accounts:
    #     account_docs = load_bluesky_data(
    #         author_handle=account,
    #         limit=10
    #     )
    #     all_documents["news"].extend(account_docs)
    #     time.sleep(2)
    

    # ═══════════════════════════════════════════════════════════════
    # RÉSUMÉ FINAL
    # ═══════════════════════════════════════════════════════════════
    print("="*60)
    print(" RÉSUMÉ DE L'INGESTION")
    print("="*60)
    
    total_docs = sum(len(docs) for docs in all_documents.values())
    
    for namespace, docs in all_documents.items():
        status = "" if len(docs) > 0 else " VIDE"
        print(f"   {namespace:20s} : {len(docs):4d} documents {status}")
    
    print("-"*60)
    print(f"  TOTAL DOCUMENTS : {total_docs}")
    print("="*60 + "\n")
    
    # Sauvegarde
    with open("ingested_documents.json", "w", encoding="utf-8") as f:
        serializable_docs = [
            {"page_content": doc.page_content, "metadata": doc.metadata}
            for docs in all_documents.values() for doc in docs
        ]
        json.dump(serializable_docs, f, ensure_ascii=False, indent=2)
    
    print("Documents sauvegardés dans 'ingested_documents.json'")
    print("Prêt pour le chunking et les embeddings (Notebook 3)")
    
    return all_documents

# ═══════════════════════════════════════════════════════════════
# EXÉCUTION
# ═══════════════════════════════════════════════════════════════

if __name__ == "__main__":
    documents_by_namespace = ingest_all_sources()
    
    # Sauvegarde
    with open("ingested_documents.json", "w") as f:
        serializable = {}
        for ns, docs in documents_by_namespace.items():
            serializable[ns] = [
                {
                    "page_content": doc.page_content,
                    "metadata": doc.metadata
                }
                for doc in docs
            ]
        json.dump(serializable, f, indent=2)
    
    print(" Documents sauvegardés dans 'ingested_documents.json'")
    print(" Prêt pour le chunking et les embeddings (Notebook 3)")

    #Yfinace ne marche pas comme il faut,
    # Opencorporates et crunchbase ne marchent pas non plus 
    # Reddit bloqué recommencer plus tard.

 Configuration des sources initialisée

 DÉMARRAGE DE L'INGESTION MULTI-SOURCES

 Chargement SEC EDGAR...
    Métadonnées : Apple Inc.
[2026-01-29T17:13:01.704521] SEC_EDGAR - INFO : Requête pour CIK 0000320193
[2026-01-29T17:13:02.185533] SEC_EDGAR - SUCCESS : 1 documents trouvés pour Apple Inc.
    Métadonnées : Microsoft Corp.
[2026-01-29T17:13:03.190284] SEC_EDGAR - INFO : Requête pour CIK 0000789019
[2026-01-29T17:13:04.535218] SEC_EDGAR - SUCCESS : 1 documents trouvés pour MICROSOFT CORP
    Métadonnées : Alphabet Inc. (Google)
[2026-01-29T17:13:05.635522] SEC_EDGAR - INFO : Requête pour CIK 0001652044
[2026-01-29T17:13:06.188902] SEC_EDGAR - SUCCESS : 1 documents trouvés pour Alphabet Inc.
    Métadonnées : Amazon.com Inc.
[2026-01-29T17:13:07.198172] SEC_EDGAR - INFO : Requête pour CIK 0001018724
[2026-01-29T17:13:07.813886] SEC_EDGAR - SUCCESS : 1 documents trouvés pour AMAZON COM INC
    Métadonnées : Meta Platforms Inc.
[2026-01-29T17:13:08.821764] SEC_EDGAR - INFO : Requête

Finished call to 'dbnomics._fetch_response' after 0.537(s), this was the 1st time calling it.
Finished call to 'dbnomics._fetch_response' after 1.877(s), this was the 2nd time calling it.
Finished call to 'dbnomics._fetch_response' after 3.178(s), this was the 3rd time calling it.


[2026-01-29T17:19:35.658429] DBNOMICS - ERROR : Inflation France - IPC (mensuel): Could not fetch data from URL 'https://api.db.nomics.world/v22/series/OECD/MEI/FRA.CP.IXOBSA.M?observations=1&offset=0'
[2026-01-29T17:19:35.659408] DBNOMICS - INFO : Fetch: Taux de chômage France (mensuel)


Finished call to 'dbnomics._fetch_response' after 0.528(s), this was the 1st time calling it.
Finished call to 'dbnomics._fetch_response' after 1.805(s), this was the 2nd time calling it.
Finished call to 'dbnomics._fetch_response' after 3.490(s), this was the 3rd time calling it.


[2026-01-29T17:19:39.171893] DBNOMICS - ERROR : Taux de chômage France (mensuel): Could not fetch data from URL 'https://api.db.nomics.world/v22/series/OECD/MEI/FRA.LR.STSA.M?observations=1&offset=0'
[2026-01-29T17:19:39.174184] DBNOMICS - INFO : Fetch: Indicateur composite avancé France (Lead Indicator)


...

[2026-01-29T17:19:44.879352] DBNOMICS - ERROR : Confiance des consommateurs France: Could not fetch data from URL 'https://api.db.nomics.world/v22/series/OECD/MEI/FRA.CSC.GOLD.IXOBSA.M?observations=1&offset=0'
[2026-01-29T17:19:44.880272] DBNOMICS - INFO : Fetch: Confiance des entreprises France (Business Climate)


Finished call to 'dbnomics._fetch_response' after 0.538(s), this was the 1st time calling it.
Finished call to 'dbnomics._fetch_response' after 1.921(s), this was the 2nd time calling it.
Finished call to 'dbnomics._fetch_response' after 4.427(s), this was the 3rd time calling it.


[2026-01-29T17:19:49.309381] DBNOMICS - ERROR : Confiance des entreprises France (Business Climate): Could not fetch data from URL 'https://api.db.nomics.world/v22/series/OECD/MEI/FRA.BSC.GOLD.IXOBSA.M?observations=1&offset=0'
[2026-01-29T17:19:49.315682] DBNOMICS - INFO : Fetch: Taux de prêt bancaire aux entreprises (Zone Euro)
[2026-01-29T17:19:51.264152] DBNOMICS - INFO : Fetch: Taux long terme (10 ans) France
[2026-01-29T17:19:54.047569] DBNOMICS - INFO : Fetch: Ventes de détail France (Consommation)
[2026-01-29T17:19:55.526528] DBNOMICS - INFO : Fetch: Production industrielle France (Général)
[2026-01-29T17:19:57.255207] DBNOMICS - INFO : Fetch: Production dans la Construction (Immobilier)
[2026-01-29T17:19:59.513848] DBNOMICS - INFO : Fetch: PIB USA (trimestriel)
[2026-01-29T17:20:00.732227] DBNOMICS - INFO : Fetch: PIB Allemagne (trimestriel)
[2026-01-29T17:20:01.898898] DBNOMICS - INFO : Fetch: Inflation USA (mensuel)


Finished call to 'dbnomics._fetch_response' after 0.383(s), this was the 1st time calling it.
Finished call to 'dbnomics._fetch_response' after 1.637(s), this was the 2nd time calling it.
Finished call to 'dbnomics._fetch_response' after 2.387(s), this was the 3rd time calling it.


[2026-01-29T17:20:04.288718] DBNOMICS - ERROR : Inflation USA (mensuel): Could not fetch data from URL 'https://api.db.nomics.world/v22/series/OECD/MEI/USA.CP.IXOBSA.M?observations=1&offset=0'
[2026-01-29T17:20:04.289947] DBNOMICS - SUCCESS : 8 séries récupérées
   8 séries économiques chargées.

[2026-01-29T17:20:04.291183] ADZUNA - INFO : Requête search: what='Senior Manager Audit', where='Paris', country=fr
[2026-01-29T17:20:05.353945] ADZUNA - INFO : Trouvé 9 offres (moyenne salaire: 48000€)
[2026-01-29T17:20:05.355099] ADZUNA - SUCCESS : 9 documents créés
[2026-01-29T17:20:06.361257] ADZUNA - INFO : Requête search: what='Consultant M&A Transaction Services', where='France', country=fr
[2026-01-29T17:20:07.082725] ADZUNA - INFO : Trouvé 0 offres (moyenne salaire: 0€)
[2026-01-29T17:20:07.083321] ADZUNA - SUCCESS : 0 documents créés
[2026-01-29T17:20:08.091425] ADZUNA - INFO : Requête search: what='Manager Cybersecurité', where='Île-de-France', country=fr
[2026-01-29T17:20:09.493249

In [28]:
"""
NOTEBOOK 3 : Chunking Adaptatif & Embeddings Mistral
====================================================

OBJECTIF : Découper les documents de manière intelligente
           et générer des embeddings optimisés pour Pinecone.

RÉFÉRENCES :
- LangChain Text Splitters : https://python.langchain.com/docs/modules/data_connection/document_transformers/
- Mistral Embeddings : https://docs.mistral.ai/capabilities/embeddings/
- Chunking Best Practices : https://www.pinecone.io/learn/chunking-strategies/

MÉTHODOLOGIE :
1. Chunking adaptatif selon le type de document
2. Enrichissement des métadonnées
3. Génération d'embeddings par batch (optimisation)
4. Validation de la qualité
"""

import os
import json
from typing import List, Dict
from dotenv import load_dotenv

# LangChain
from langchain_core.documents import Document
from langchain_text_splitters import (
    RecursiveCharacterTextSplitter,
    HTMLHeaderTextSplitter
)
from langchain_mistralai import MistralAIEmbeddings
from langchain_mistralai import ChatMistralAI  # NOUVEAU pour FACT-RAG

load_dotenv()

MISTRAL_API_KEY= os.getenv("MISTRAL_API_KEY")

# ═══════════════════════════════════════════════════════════════
# SECTION 1 : STRATÉGIES DE CHUNKING PAR TYPE DE DOCUMENT
# ═══════════════════════════════════════════════════════════════

"""
JUSTIFICATION DES CHOIX DE CHUNKING

D'après vos notes (KPMG v2.pdf), le chunking est l'étape la plus sous-estimée
mais déterminante pour la qualité du RAG.

PRINCIPES :
1. Granularité adaptée au contenu
   - Petits chunks (500 chars) : KPIs, chiffres précis
   - Gros chunks (1000+ chars) : analyses, raisonnements

2. Overlap significatif (15-20%)
   - Évite de couper les informations critiques
   - Maintient la cohérence contextuelle

3. Séparateurs intelligents
   - Paragraphes > Phrases > Mots
   - Préserve la structure sémantique
"""

class AdaptiveChunker:
    """Découpe intelligente selon le type de document"""
    
    def __init__(self):
        # Splitter pour rapports financiers (SEC, yfinance)
        self.financial_splitter = RecursiveCharacterTextSplitter(
            chunk_size=800,      # Balance entre détail et contexte
            chunk_overlap=150,   # ~19% overlap
            separators=["\n\n", "\n", ". ", " ", ""],
            add_start_index=True  # Traçabilité dans le document source
        )
        
        # Splitter pour actualités (courtes, denses)
        self.news_splitter = RecursiveCharacterTextSplitter(
            chunk_size=500,      # Articles courts
            chunk_overlap=100,   # 20% overlap
            separators=["\n\n", "\n", ". ", " ", ""],
            add_start_index=True
        )
        
        # Splitter HTML (communiqués de presse structurés)
        self.html_splitter = HTMLHeaderTextSplitter(
            headers_to_split_on=[
                ("h1", "Header 1"),
                ("h2", "Header 2"),
                ("h3", "Header 3"),
            ]
        )
    
    def chunk_documents(self, documents: List[Document], namespace: str) -> List[Document]:
        """
        Applique la stratégie de chunking appropriée selon le namespace
        """
        print(f"\n Chunking pour namespace '{namespace}'...")
        
        # Mapping namespace → splitter
        if namespace in ["financial_reports", "macro_data", "startups"]:
            # Documents denses, analytiques, avec chiffres
            splitter = self.financial_splitter
        elif namespace in ["news", "social_signals"]:
            # Contenu court, rapide à consommer
            splitter = self.news_splitter
        elif namespace == "facts":
            # Facts déjà structurés, pas de chunking nécessaire
            return documents  # Retour direct sans chunking
        elif namespace == "web_quarantine":
            # Sources web externes, traitement léger
            splitter = self.news_splitter
        else:
            # Défaut conservateur
            splitter = self.news_splitter
        
        # Découpage
        chunked_docs = []
        for doc in documents:
            chunks = splitter.split_documents([doc])
            
            # Enrichir métadonnées
            for i, chunk in enumerate(chunks):
                chunk.metadata.update({
                    "chunk_index": i,
                    "total_chunks": len(chunks),
                    "chunking_strategy": splitter.__class__.__name__
                })
                chunked_docs.append(chunk)
        
        print(f"    {len(documents)} docs → {len(chunked_docs)} chunks")
        return chunked_docs

chunker = AdaptiveChunker()
print(" Chunker adaptatif initialisé")

# ═══════════════════════════════════════════════════════════════
# SECTION 2 : CHARGEMENT DES DOCUMENTS INGÉRÉS
# ═══════════════════════════════════════════════════════════════

"""
Récupération des documents depuis le Notebook 2
"""

def load_ingested_documents() -> Dict[str, List[Document]]:
    """Charge les documents depuis le fichier JSON"""
    try:
        with open("ingested_documents.json", "r") as f:
            data = json.load(f)
        
        # Reconversion en objets Document
        documents_by_ns = {}
        for namespace, docs_data in data.items():
            documents_by_ns[namespace] = [
                Document(
                    page_content=d["page_content"],
                    metadata=d["metadata"]
                )
                for d in docs_data
            ]
        
        print(f" {sum(len(docs) for docs in documents_by_ns.values())} documents chargés")
        return documents_by_ns
    
    except FileNotFoundError:
        print(" Fichier 'ingested_documents.json' introuvable.")
        print("   Exécutez d'abord le Notebook 2 (ingestion)")
        return {}

docs_by_namespace = load_ingested_documents()

# ═══════════════════════════════════════════════════════════════
# SECTION 3 : APPLICATION DU CHUNKING
# ═══════════════════════════════════════════════════════════════

"""
Application du chunking adaptatif sur tous les namespaces
"""

chunked_by_namespace = {}

print("\n" + "="*60)
print(" PHASE DE CHUNKING ADAPTATIF")
print("="*60)

for namespace, documents in docs_by_namespace.items():
    if documents:
        chunked_docs = chunker.chunk_documents(documents, namespace)
        chunked_by_namespace[namespace] = chunked_docs

print("\n Résumé du chunking :")
for ns, chunks in chunked_by_namespace.items():
    print(f"   {ns}: {len(chunks)} chunks")

# ═══════════════════════════════════════════════════════════════
# SECTION 3B : EXTRACTION DE FACTS (FACT-RAG "LITE")
# ═══════════════════════════════════════════════════════════════
"""
Objectif :
- Extraire des "facts" structurés (chiffres / événements) à partir des chunks déjà produits
- Les stocker comme Documents LangChain dans un namespace dédié : "facts"
- Les facts seront ensuite embed + indexés dans Pinecone comme les autres chunks

⚠️ Version "Lite" (hackathon) :
- On limite volontairement le nombre d'appels LLM
- On extrait uniquement quelques types de facts high-signal
"""

from typing import Optional
from langchain_mistralai import ChatMistralAI

FACT_NAMESPACE = "facts"

FACT_TYPES = [
    "deal",              # acquisition / partnership / funding
    "financial_kpi",      # revenue, margin, capex, guidance
    "market_size",        # taille de marché
    "growth_rate",        # croissance / CAGR
    "regulation_event",   # régulation / amende / décision
    "product_launch"      # lancement / pivot stratégique
]

# Limites anti-explosion de coûts (hackathon-friendly)
MAX_CHUNKS_PER_NAMESPACE_FOR_FACTS = 10
MAX_CHARS_PER_CHUNK_FOR_FACTS = 1200

def _safe_json_loads(text: str) -> Optional[dict]:
    """Parse JSON robuste : récupère le premier bloc JSON trouvable."""
    try:
        return json.loads(text)
    except Exception:
        pass

    # fallback : tenter d'extraire le 1er objet JSON dans le texte
    import re
    m = re.search(r"\{.*\}", text, flags=re.DOTALL)
    if m:
        try:
            return json.loads(m.group(0))
        except Exception:
            return None
    return None

def extract_facts_from_chunks(chunked_by_namespace: Dict[str, List[Document]]) -> List[Document]:
    """
    Extrait des facts depuis un échantillon de chunks (par namespace).
    Retourne une liste de Documents (facts) qui seront indexés dans Pinecone.
    """
    if not MISTRAL_API_KEY:
        print(" MISTRAL_API_KEY manquante → extraction FACT ignorée.")
        return []

    llm = ChatMistralAI(
        model="mistral-small",
        mistral_api_key=MISTRAL_API_KEY,
        temperature=0
    )

    fact_docs: List[Document] = []

    # On cible les namespaces les plus pertinents pour des facts "actionnables"
    # Namespaces pertinents pour extraction de facts
    FACT_EXTRACTION_NAMESPACES = [
        "financial_reports",  # Priorité : chiffres financiers
        "news",               # Deals, événements
        "macro_data",         # KPIs macro
        "startups"            # Funding rounds, acquisitions
    ]

    target_namespaces = [ns for ns in FACT_EXTRACTION_NAMESPACES if ns in chunked_by_namespace]
    print("\n" + "="*60)
    print(" PHASE FACTS (LITE) — Extraction structurée")
    print("="*60)

    for ns in target_namespaces:
        chunks = chunked_by_namespace.get(ns) or []
        if not chunks:
            continue

        sample = chunks[:MAX_CHUNKS_PER_NAMESPACE_FOR_FACTS]

        # On construit un "mini-dossier" pour limiter à 1 appel LLM / namespace
        dossier_parts = []
        for i, d in enumerate(sample, start=1):
            url = d.metadata.get("url") or d.metadata.get("source") or ""
            published = d.metadata.get("published_at") or d.metadata.get("date") or ""
            text = (d.page_content or "")[:MAX_CHARS_PER_CHUNK_FOR_FACTS]
            dossier_parts.append(f"CHUNK {i}\nURL: {url}\nDATE: {published}\nTEXTE:\n{text}\n")

        dossier = "\n---\n".join(dossier_parts)

        prompt = f"""
Tu es un analyste en veille stratégique (cabinet de conseil).
À partir des CHUNKS ci-dessous, extrais des FACTS structurés.

Règles :
- Retourne UNIQUEMENT du JSON valide (pas de Markdown).
- Les facts doivent être vérifiables dans le texte.
- Si tu n'es pas sûr, n'invente pas.
- Maximum 12 facts.

Schéma JSON :
{{
  "facts": [
    {{
      "fact_type": one_of({FACT_TYPES}),
      "entity": "Nom d'entreprise / marché / organisme",
      "value": "nombre ou texte court (optionnel)",
      "unit": "USD|EUR|%|... (optionnel)",
      "period": "FY2023|Q3 2024|2025-01-15|... (optionnel)",
      "geography": "US|EU|France|Global|... (optionnel)",
      "source_url": "URL à citer",
      "source_date": "date si disponible",
      "quote_span": "extrait EXACT du chunk qui justifie le fact"
    }}
  ]
}}

CHUNKS (namespace={ns}) :
{dossier}
"""
        try:
            llm_out = llm.invoke(prompt)
            parsed = _safe_json_loads(getattr(llm_out, "content", str(llm_out)))

            if not parsed or "facts" not in parsed:
                print(f"⚠️ Aucun JSON FACT valide pour namespace '{ns}'")
                continue

            ns_facts = parsed.get("facts") or []
            print(f" {len(ns_facts)} facts extraits depuis '{ns}'")

            for f in ns_facts:
                # Normalisation minimale
                fact_type = f.get("fact_type", "").strip()
                entity = (f.get("entity") or "").strip()
                source_url = (f.get("source_url") or "").strip()
                quote = (f.get("quote_span") or "").strip()

                if not fact_type or not entity or not source_url or not quote:
                    continue

                # Texte compact pour embedding/retrieval
                summary = f"[{fact_type}] {entity} — {f.get('value','')} {f.get('unit','')}".strip()
                if f.get("period"):
                    summary += f" (période: {f.get('period')})"
                if f.get("geography"):
                    summary += f" (geo: {f.get('geography')})"

                fact_docs.append(Document(
                    page_content=summary,
                    metadata={
                        "fact_type": fact_type,
                        "entity": entity,
                        "value": f.get("value"),
                        "unit": f.get("unit"),
                        "period": f.get("period"),
                        "geography": f.get("geography"),
                        "source_url": source_url,
                        "source_date": f.get("source_date"),
                        "quote_span": quote,
                        "source_namespace": ns,
                        "doc_type": "fact"
                    }
                ))

        except Exception as e:
            print(f" Erreur extraction FACT sur namespace '{ns}': {e}")
            continue

    # Sauvegarde locale (MVP)
    if fact_docs:
        serializable = [
            {"page_content": d.page_content, "metadata": d.metadata}
            for d in fact_docs
        ]
        with open("facts.json", "w", encoding="utf-8") as f:
            json.dump(serializable, f, ensure_ascii=False, indent=2)
        print(" Facts sauvegardés dans 'facts.json'")

    return fact_docs

# Exécution : on extrait des facts puis on les ajoute au pipeline pour embeddings + indexation
fact_documents = extract_facts_from_chunks(chunked_by_namespace)
if fact_documents:
    chunked_by_namespace[FACT_NAMESPACE] = fact_documents
    print(f" Ajout de {len(fact_documents)} facts dans chunked_by_namespace['{FACT_NAMESPACE}']")
else:
    print("Aucun fact ajouté (soit aucun fact trouvé, soit extraction désactivée)")


# ═══════════════════════════════════════════════════════════════
# SECTION 4 : GÉNÉRATION DES EMBEDDINGS MISTRAL
# ═══════════════════════════════════════════════════════════════

"""
CONFIGURATION MISTRAL EMBEDDINGS

Modèle : mistral-embed
Dimension : 1024
Coût : Gratuit avec limits (voir plan Mistral)

RÉFÉRENCE :
https://docs.mistral.ai/capabilities/embeddings/

OPTIMISATION :
- Traitement par batch pour limiter les appels API
- Cache local des embeddings (évite recalcul)
- Gestion des erreurs et retry
"""

MISTRAL_API_KEY = os.getenv("MISTRAL_API_KEY")

if not MISTRAL_API_KEY:
    raise ValueError(" MISTRAL_API_KEY manquante dans .env")

embeddings_model = MistralAIEmbeddings(
    model="mistral-embed",
    mistral_api_key=MISTRAL_API_KEY
)

print("\n Modèle Mistral-embed initialisé")

def generate_embeddings_batch(documents: List[Document], batch_size: int = 50) -> List[Document]:
    """
    Génère les embeddings par batch pour optimiser les appels API
    
    Args:
        documents: Documents à embedder
        batch_size: Nombre de documents par batch
    
    Returns:
        Documents avec embeddings dans les métadonnées
    """
    print(f"\n Génération de {len(documents)} embeddings...")
    
    total_batches = (len(documents) + batch_size - 1) // batch_size
    
    for i in range(0, len(documents), batch_size):
        batch = documents[i:i+batch_size]
        batch_num = i // batch_size + 1
        
        try:
            # Extraire les textes
            texts = [doc.page_content for doc in batch]
            
            # Générer les embeddings
            embeddings = embeddings_model.embed_documents(texts)
            
            # Ajouter les embeddings aux métadonnées
            for doc, embedding in zip(batch, embeddings):
                doc.metadata["embedding"] = embedding
            
            print(f"    Batch {batch_num}/{total_batches} traité")
        
        except Exception as e:
            print(f"    Erreur batch {batch_num}: {e}")
            # Continuer avec les autres batches
            continue
    
    return documents

# ═══════════════════════════════════════════════════════════════
# SECTION 5 : VALIDATION DE LA QUALITÉ
# ═══════════════════════════════════════════════════════════════

"""
MÉTRIQUES DE QUALITÉ

Avant d'envoyer vers Pinecone, on valide :
1. Tous les chunks ont des embeddings
2. Les dimensions sont correctes (1024)
3. Les métadonnées sont complètes
"""

def validate_chunked_documents(docs_by_ns: Dict[str, List[Document]]) -> bool:
    """Valide la qualité des documents chunkés et embeddés"""
    print("\n" + "="*60)
    print(" VALIDATION DE LA QUALITÉ")
    print("="*60)
    
    # ═══════════════════════════════════════════════════════════
    # VÉRIFICATION : Tous les namespaces attendus sont présents ?
    # ═══════════════════════════════════════════════════════════
    EXPECTED_NAMESPACES = [
        "financial_reports", "news", "macro_data", 
        "startups", "social_signals","startups"
        # "facts" et "web_quarantine" sont optionnels
    ]
    
    missing_namespaces = [ns for ns in EXPECTED_NAMESPACES if ns not in docs_by_ns or not docs_by_ns[ns]]
    if missing_namespaces:
        print(f"\n  Namespaces manquants ou vides : {missing_namespaces}")
        print("   (Normal si certaines sources n'ont pas été activées)")
    
    total_chunks = sum(len(docs) for docs in docs_by_ns.values())
    chunks_with_embeddings = 0
    invalid_embeddings = []
    
    for namespace, documents in docs_by_ns.items():
        print(f"\n Namespace: {namespace}")
        
        for i, doc in enumerate(documents):
            # Vérifier présence embedding
            if "embedding" in doc.metadata:
                chunks_with_embeddings += 1
                
                # Vérifier dimension
                emb_dim = len(doc.metadata["embedding"])
                if emb_dim != 1024:
                    invalid_embeddings.append((namespace, i, emb_dim))
            
            # Vérifier métadonnées essentielles
            required_metadata = ["source", "namespace"]
            missing = [key for key in required_metadata if key not in doc.metadata]
            if missing:
                print(f"     Chunk {i} : métadonnées manquantes {missing}")
        
        print(f"    {len(documents)} chunks validés")
    
    # Rapport final
    print("\n" + "="*60)
    print(f" Chunks avec embeddings : {chunks_with_embeddings}/{total_chunks}")
    
    if invalid_embeddings:
        print(f"  Embeddings invalides (dimension ≠ 1024) : {len(invalid_embeddings)}")
        for ns, idx, dim in invalid_embeddings[:5]:  # Afficher les 5 premiers
            print(f"   - {ns}[{idx}] : dimension {dim}")
    else:
        print(" Toutes les dimensions sont correctes (1024)")
    
    print("="*60)
    
    return chunks_with_embeddings == total_chunks

# ═══════════════════════════════════════════════════════════════
# SECTION 6 : PIPELINE COMPLET
# ═══════════════════════════════════════════════════════════════

"""
Orchestration complète : Chunking → Embeddings → Validation
"""

def process_all_documents():
    """Pipeline complet de traitement"""
    
    # Étape 1 : Chunking (déjà fait)
    print("\n Étape 1 : Chunking terminé")
    
    # ═══════════════════════════════════════════════════════════
    # ÉTAPE 1B : EXTRACTION DE FACTS (NOUVEAU)
    # ═══════════════════════════════════════════════════════════
    print("\n" + "="*60)
    print(" ÉTAPE 1B : EXTRACTION DE FACTS STRUCTURÉS")
    print("="*60)
    
    fact_documents = extract_facts_from_chunks(chunked_by_namespace)
    
    if fact_documents:
        chunked_by_namespace[FACT_NAMESPACE] = fact_documents
        print(f" {len(fact_documents)} facts ajoutés au namespace '{FACT_NAMESPACE}'")
    else:
        print(" Aucun fact extrait (soit aucun trouvé, soit extraction désactivée)")
    
    # Étape 2 : Génération des embeddings
    print("\n" + "="*60)
    print("ÉTAPE 2 : GÉNÉRATION DES EMBEDDINGS")
    print("="*60)
    
    for namespace, documents in chunked_by_namespace.items():
        if documents:
            print(f"\n Traitement namespace '{namespace}'...")
            generate_embeddings_batch(documents)
    
    # Étape 3 : Validation
    is_valid = validate_chunked_documents(chunked_by_namespace)
    
    if is_valid:
        # Sauvegarde pour le Notebook 4
        print("\n Sauvegarde des documents pour indexation...")
        
        serializable = {}
        for ns, docs in chunked_by_namespace.items():
            serializable[ns] = [
                {
                    "page_content": doc.page_content,
                    "metadata": {
                        k: v for k, v in doc.metadata.items()
                        if k != "embedding"  # Embeddings trop gros pour JSON
                    },
                    "embedding": doc.metadata.get("embedding", [])
                }
                for doc in docs
            ]
        
        with open("embedded_documents.json", "w") as f:
            json.dump(serializable, f, indent=2)
        
        print(" Documents sauvegardés dans 'embedded_documents.json'")
        print("\n Prêt pour l'indexation Pinecone (Notebook 4)")
    else:
        print("\n Validation échouée. Vérifiez les erreurs ci-dessus.")

# Exécution
if __name__ == "__main__":
    if docs_by_namespace:
        process_all_documents()
    else:
        print("\n  Aucun document à traiter.")
        print("   Exécutez d'abord les Notebooks 1 et 2.")

 Chunker adaptatif initialisé
 1326 documents chargés

 PHASE DE CHUNKING ADAPTATIF

 Chunking pour namespace 'financial_reports'...
    40 docs → 10668 chunks

 Chunking pour namespace 'news'...
    1103 docs → 1698 chunks

 Chunking pour namespace 'macro_data'...
    88 docs → 144 chunks

 Chunking pour namespace 'social_signals'...
    95 docs → 150 chunks

 Résumé du chunking :
   financial_reports: 10668 chunks
   news: 1698 chunks
   macro_data: 144 chunks
   social_signals: 150 chunks

 PHASE FACTS (LITE) — Extraction structurée
 10 facts extraits depuis 'financial_reports'
 8 facts extraits depuis 'news'
 12 facts extraits depuis 'macro_data'
 Facts sauvegardés dans 'facts.json'
 Ajout de 30 facts dans chunked_by_namespace['facts']

 Modèle Mistral-embed initialisé

 Étape 1 : Chunking terminé

 ÉTAPE 1B : EXTRACTION DE FACTS STRUCTURÉS

 PHASE FACTS (LITE) — Extraction structurée
 10 facts extraits depuis 'financial_reports'
 7 facts extraits depuis 'news'
 12 facts extraits d

In [29]:
"""
NOTEBOOK 4 : Indexation Pinecone avec Namespaces
=================================================

OBJECTIF : Indexer les documents embeddés dans Pinecone
           en utilisant les namespaces pour l'isolation des sources.

RÉFÉRENCES :
- Pinecone Upsert : https://docs.pinecone.io/docs/upsert-data
- LangChain Pinecone : https://python.langchain.com/docs/integrations/vectorstores/pinecone
- Namespaces Best Practices : https://docs.pinecone.io/docs/namespaces

MÉTHODOLOGIE :
1. Chargement des documents embeddés
2. Conversion au format Pinecone
3. Upsert par batch et namespace
4. Validation de l'indexation
"""
"""
NOTEBOOK 4 : Indexation Pinecone avec Namespaces
=================================================

OBJECTIF : Indexer les documents embeddés dans Pinecone
           en utilisant les namespaces pour l'isolation des sources.

RÉFÉRENCES :
- Pinecone Upsert : https://docs.pinecone.io/docs/upsert-data
- LangChain Pinecone : https://python.langchain.com/docs/integrations/vectorstores/pinecone
- Namespaces Best Practices : https://docs.pinecone.io/docs/namespaces

MÉTHODOLOGIE :
1. Chargement des documents embeddés
2. Conversion au format Pinecone
3. Upsert par batch et namespace
4. Validation de l'indexation
"""

import os
import json
import time
import hashlib
from typing import List, Dict, Tuple
from dotenv import load_dotenv

# Pinecone
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain_mistralai import MistralAIEmbeddings
from langchain_core.documents import Document

load_dotenv()

# ═══════════════════════════════════════════════════════════════
# SECTION 1 : INITIALISATION
# ═══════════════════════════════════════════════════════════════

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
MISTRAL_API_KEY = os.getenv("MISTRAL_API_KEY")
INDEX_NAME = "kpmg-veille"

if not PINECONE_API_KEY or not MISTRAL_API_KEY:
    raise ValueError(" Clés API manquantes dans .env")

# Clients
pc = Pinecone(api_key=PINECONE_API_KEY)
embeddings_model = MistralAIEmbeddings(
    model="mistral-embed",
    mistral_api_key=MISTRAL_API_KEY
)

print(" Clients Pinecone et Mistral initialisés")

# ═══════════════════════════════════════════════════════════════
# SECTION 2 : CHARGEMENT DES DOCUMENTS EMBEDDÉS
# ═══════════════════════════════════════════════════════════════

def load_embedded_documents() -> Dict[str, List[Document]]:
    """Charge les documents avec leurs embeddings depuis le Notebook 3"""
    try:
        with open("embedded_documents.json", "r") as f:
            data = json.load(f)
        
        docs_by_ns = {}
        for namespace, docs_data in data.items():
            docs_by_ns[namespace] = [
                Document(
                    page_content=d["page_content"],
                    metadata={
                        **d["metadata"],
                        "embedding": d["embedding"]
                    }
                )
                for d in docs_data
            ]
        
        total = sum(len(docs) for docs in docs_by_ns.values())
        print(f" {total} documents chargés depuis embedded_documents.json")
        
        return docs_by_ns
    
    except FileNotFoundError:
        print(" Fichier 'embedded_documents.json' introuvable.")
        print("   Exécutez d'abord le Notebook 3 (chunking & embeddings)")
        return {}

docs_by_namespace = load_embedded_documents()

# ═══════════════════════════════════════════════════════════════
# SECTION 3 : PRÉPARATION DES VECTEURS POUR PINECONE
# ═══════════════════════════════════════════════════════════════

"""
FORMAT PINECONE UPSERT

Structure requise :
{
    "id": "unique_id",
    "values": [0.1, 0.2, ...],  # Embedding (liste de 1024 floats)
    "metadata": {...}            # Métadonnées (max 40 KB par vecteur)
}

RÉFÉRENCE :
https://docs.pinecone.io/docs/upsert-data
"""

def prepare_vectors_for_pinecone(documents: List[Document], namespace: str) -> List[Dict]:
    """
    Convertit les Documents au format Pinecone avec gestion des doublons
    
    ANTI-DOUBLONS : Utilise un hash MD5 du contenu pour créer des IDs déterministes.
    → Si vous relancez l'ingestion, Pinecone fera un UPDATE au lieu d'un INSERT.
    """
    vectors = []
    
    for i, doc in enumerate(documents):
        # ═══════════════════════════════════════════════════════════
        # 1. GÉNÉRATION D'ID DÉTERMINISTE (ANTI-DOUBLONS)
        # ═══════════════════════════════════════════════════════════
        # Hash MD5 du contenu → toujours le même ID pour le même texte
        content_hash = hashlib.md5(doc.page_content.encode('utf-8')).hexdigest()
        vector_id = f"{namespace}_{content_hash}"
        
        # ═══════════════════════════════════════════════════════════
        # 2. EXTRACTION DE L'EMBEDDING
        # ═══════════════════════════════════════════════════════════
        embedding = doc.metadata.pop("embedding", None)
        
        if not embedding:
            continue
        
        # ═══════════════════════════════════════════════════════════
        # 3. NETTOYAGE DES MÉTADONNÉES (LIMITE 40KB PINECONE)
        # ═══════════════════════════════════════════════════════════
        clean_metadata = {}
        
        # Liste blanche : métadonnées à conserver
        allowed_keys = [
            "source", "namespace", "title", "url", "published_at",
            "company", "company_name", "cik", "form_type", "filing_date",
            "author", "source_name", "ticker", "series_id", "siren",
            "search_query", "author_handle", "created_at"
        ]
        
        for key in allowed_keys:
            if key in doc.metadata:
                value = doc.metadata[key]
                
                # Conversion en string si nécessaire
                if isinstance(value, (list, dict)):
                    clean_metadata[key] = str(value)[:500]  # Limiter taille
                elif value is not None:
                    clean_metadata[key] = str(value)[:500]
        
        # Ajouter un extrait du texte pour prévisualisation
        clean_metadata["text"] = doc.page_content[:1000]
        
        # ═══════════════════════════════════════════════════════════
        # 4. CRÉATION DU VECTEUR PINECONE
        # ═══════════════════════════════════════════════════════════
        vector = {
            "id": vector_id,           # ID déterministe (anti-doublons)
            "values": embedding,       # Vecteur 1024 dimensions
            "metadata": clean_metadata # Métadonnées nettoyées
        }
        
        vectors.append(vector)
    
    return vectors
# ═══════════════════════════════════════════════════════════════
# SECTION 4 : UPSERT PAR BATCH ET NAMESPACE
# ═══════════════════════════════════════════════════════════════

"""
STRATÉGIE D'UPSERT

1. Traiter par batch de 100 vecteurs (limite Pinecone)
2. Utiliser les namespaces pour isoler les sources
3. Gérer les erreurs et retry

RÉFÉRENCE :
https://docs.pinecone.io/docs/upsert-data#batching-upserts
"""

def upsert_to_pinecone(vectors: List[Dict], namespace: str, batch_size: int = 100):
    """
    Envoie les vecteurs vers Pinecone par batch
    
    Args:
        vectors: Vecteurs au format Pinecone
        namespace: Namespace de destination
        batch_size: Taille des batches
    """
    index = pc.Index(INDEX_NAME)
    total_batches = (len(vectors) + batch_size - 1) // batch_size
    
    print(f"\n Upsert vers namespace '{namespace}'...")
    print(f"   {len(vectors)} vecteurs en {total_batches} batches")
    
    for i in range(0, len(vectors), batch_size):
        batch = vectors[i:i+batch_size]
        batch_num = i // batch_size + 1
        
        try:
            index.upsert(
                vectors=batch,
                namespace=namespace
            )
            print(f"   Batch {batch_num}/{total_batches} envoyé")
            
            # Rate limiting (10 req/sec max pour Pinecone gratuit)
            time.sleep(0.1)
        
        except Exception as e:
            print(f"    Erreur batch {batch_num}: {e}")
            # Retry une fois
            try:
                time.sleep(1)
                index.upsert(vectors=batch, namespace=namespace)
                print(f"   Retry réussi pour batch {batch_num}")
            except Exception as e2:
                print(f"    Retry échoué : {e2}")
                continue

# ═══════════════════════════════════════════════════════════════
# SECTION 5 : INDEXATION COMPLÈTE
# ═══════════════════════════════════════════════════════════════

def index_all_documents():
    """Pipeline complet d'indexation"""
    
    print("\n" + "="*60)
    print(" INDEXATION PINECONE")
    print("="*60)
    
    if not docs_by_namespace:
        print(" Aucun document à indexer")
        return
    
    for namespace, documents in docs_by_namespace.items():
        if not documents:
            continue
        
        print(f"\n Traitement namespace : {namespace}")
        
        # Préparation des vecteurs
        vectors = prepare_vectors_for_pinecone(documents, namespace)
        
        if vectors:
            # Upsert vers Pinecone
            upsert_to_pinecone(vectors, namespace)
        else:
            print(f"     Aucun vecteur valide pour {namespace}")
    
    print("\n" + "="*60)
    print(" INDEXATION TERMINÉE")
    print("="*60)

# ═══════════════════════════════════════════════════════════════
# SECTION 6 : VALIDATION POST-INDEXATION
# ═══════════════════════════════════════════════════════════════

"""
VALIDATION :
- Vérifier le nombre de vecteurs indexés
- Tester une recherche simple
- Confirmer l'isolation des namespaces
"""

def validate_indexation():
    """Valide que l'indexation s'est bien passée"""
    
    print("\n" + "="*60)
    print(" VALIDATION DE L'INDEXATION")
    print("="*60)
    
    index = pc.Index(INDEX_NAME)
    stats = index.describe_index_stats()
    
    print(f"\n Statistiques de l'index '{INDEX_NAME}' :")
    print(f"   Total vecteurs : {stats.total_vector_count}")
    print(f"\n Vecteurs par namespace :")
    
    for namespace, info in stats.namespaces.items():
        print(f"   - {namespace}: {info.vector_count} vecteurs")

# ═══════════════════════════════════════════════════════════════
# SECTION 7 : EXÉCUTION
# ═══════════════════════════════════════════════════════════════

if __name__ == "__main__":
    # Indexation
    index_all_documents()
    
    # Validation
    time.sleep(2)  # Laisser Pinecone finaliser l'indexation
    validate_indexation()
    
    print("\n Système RAG prêt pour les requêtes (Notebook 5)")

 Clients Pinecone et Mistral initialisés
 12689 documents chargés depuis embedded_documents.json

 INDEXATION PINECONE

 Traitement namespace : financial_reports

 Upsert vers namespace 'financial_reports'...
   10668 vecteurs en 107 batches
   Batch 1/107 envoyé
   Batch 2/107 envoyé
   Batch 3/107 envoyé
   Batch 4/107 envoyé
   Batch 5/107 envoyé
   Batch 6/107 envoyé
   Batch 7/107 envoyé
   Batch 8/107 envoyé
   Batch 9/107 envoyé
   Batch 10/107 envoyé
   Batch 11/107 envoyé
   Batch 12/107 envoyé
   Batch 13/107 envoyé
   Batch 14/107 envoyé
   Batch 15/107 envoyé
   Batch 16/107 envoyé
   Batch 17/107 envoyé
   Batch 18/107 envoyé
   Batch 19/107 envoyé
   Batch 20/107 envoyé
   Batch 21/107 envoyé
   Batch 22/107 envoyé
   Batch 23/107 envoyé
   Batch 24/107 envoyé
   Batch 25/107 envoyé
   Batch 26/107 envoyé
   Batch 27/107 envoyé
   Batch 28/107 envoyé
   Batch 29/107 envoyé
   Batch 30/107 envoyé
   Batch 31/107 envoyé
   Batch 32/107 envoyé
   Batch 33/107 envoyé
   Batch

Faut il Nettoyer les métadonnées avant d’upser ?

Pour identifier l'erreur, donner le requirement, l'env et le notebook a claude 

In [30]:
#============================================================
#RAG AGENTIC : SCORE
#============================================================#
from config import PRIMARY_SOURCES
from config import SECONDARY_SOURCES
from config import SPAM_INDICATORS

def score_source(url: str) -> int:
    """
    Scoring KPMG-grade des sources web
    
    Retourne :
    - 3 : Source primaire (officielle, gouvernementale, académique)
    - 2 : Source secondaire fiable (presse reconnue, institutionnels)
    - 1 : Source tertiaire (blogs, forums)
    - 0 : Source non fiable (spam, domaines suspects)
    """
    if not url:
        return 0
    
    url = url.lower()
    
    # ═══════════════════════════════════════════════════════════
    # NIVEAU 3 : SOURCES PRIMAIRES * * *
    # ═══════════════════════════════════════════════════════════

    if any(domain in url for domain in PRIMARY_SOURCES):
        return 3
    
    # ═══════════════════════════════════════════════════════════
    # NIVEAU 2 : SOURCES SECONDAIRES FIABLES * *
    # ═══════════════════════════════════════════════════════════

    
    if any(domain in url for domain in SECONDARY_SOURCES):
        return 2
    
    # ═══════════════════════════════════════════════════════════
    # NIVEAU 0 : SOURCES NON FIABLES (BLACKLIST) 
    # ═══════════════════════════════════════════════════════════
    
    if any(indicator in url for indicator in SPAM_INDICATORS):
        return 0
    
    # ═══════════════════════════════════════════════════════════
    # NIVEAU 1 : SOURCES TERTIAIRES (PAR DÉFAUT) *
    # ═══════════════════════════════════════════════════════════
    # Blogs, forums, sites inconnus
    return 1


In [31]:
"""
NOTEBOOK 5 : RAG Query & Prompt Engineering pour Veille KPMG
============================================================

OBJECTIF : Créer un système de requêtes RAG optimisé pour la veille stratégique
           avec prompting avancé et citations obligatoires.

RÉFÉRENCES :
- Mistral Prompting : https://docs.mistral.ai/guides/prompting_capabilities/
- LangChain RAG : https://python.langchain.com/docs/use_cases/question_answering/
- KPMG Requirements : hackathon KPMG (1).pdf

EXIGENCES CRITIQUES :
✓ Citations systématiques des sources
✓ Indication de fiabilité et date
✓ Réponses structurées (pas de bullet points sauf demande explicite)
✓ IA explicable (chaîne de raisonnement)
"""

import os
from typing import List, Dict, Optional
from dotenv import load_dotenv
load_dotenv()

# LangChain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_mistralai import ChatMistralAI
from langchain_pinecone import PineconeVectorStore
from langchain_mistralai import MistralAIEmbeddings

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.documents import Document
from pinecone import Pinecone
import hashlib

web_search_tool = TavilySearchResults(k=3)
WEB_QUARANTINE_NAMESPACE = "web_quarantine"


load_dotenv()

# ═══════════════════════════════════════════════════════════════
# SECTION 1 : INITIALISATION DES COMPOSANTS RAG
# ═══════════════════════════════════════════════════════════════

MISTRAL_API_KEY = os.getenv("MISTRAL_API_KEY")
INDEX_NAME = "kpmg-veille"

# Modèle d'embeddings
embeddings = MistralAIEmbeddings(
    model="mistral-embed",
    mistral_api_key=MISTRAL_API_KEY
)

# Modèle LLM (Mistral Medium pour raisonnement)
llm = ChatMistralAI(
    model="mistral-medium",
    temperature=0,  # Déterministe pour analyses factuelles
    mistral_api_key=MISTRAL_API_KEY
)

print(" Modèles Mistral initialisés")

def judge_context_sufficiency(question: str, docs: List, llm) -> bool:
    """
    Juge si le contexte interne suffit pour répondre
    selon les standards KPMG (fiabilité, fraîcheur, complétude).
    """
    if not docs:
        return False

    joined_docs = "\n\n".join([doc.page_content for doc in docs])

    prompt = f"""
    Tu es un Senior Manager KPMG.

    QUESTION :
    {question}

    CONTEXTE DISPONIBLE :
    {joined_docs}

    Réponds STRICTEMENT par OUI ou NON.

    Le contexte permet-il de répondre de manière :
    - fiable
    - récente
    - sans extrapolation ?
    """

    verdict = llm.invoke(prompt)
    return "OUI" in verdict.content.upper()

def has_reliable_sources(docs, min_confidence=2):
    """
    Vérifie s'il existe au moins une source web fiable
    
    Args:
        docs : Liste de documents
        min_confidence : Seuil de confiance
            - 3 : Uniquement sources primaires (très strict)
            - 2 : Sources secondaires acceptées (standard KPMG)
            - 1 : Toutes sources (permissif, déconseillé)
    """
    for doc in docs:
        if (
            doc.metadata.get("origin") == "web"
            and doc.metadata.get("confidence", 0) >= min_confidence
        ):
            return True
    return False


# ═══════════════════════════════════════════════════════════════
# SECTION 2 : RETRIEVERS PAR NAMESPACE
# ═══════════════════════════════════════════════════════════════

"""
STRATÉGIE DE RETRIEVAL

On crée un retriever par namespace pour permettre des requêtes ciblées.
L'utilisateur peut spécifier le namespace ou interroger tous les namespaces.

PARAMÈTRES :
- k=5 : Top 5 documents les plus pertinents
- score_threshold : Filtrage par similarité (optionnel)

RÉFÉRENCE :
https://python.langchain.com/docs/modules/data_connection/retrievers/
"""

NAMESPACES = [
    "financial_reports",
    "news",
    "macro_data",
    "facts",
    "startups",
    "social_signals",
    "web_quarantine"
]


def get_retriever(namespace: Optional[str] = None, k: int = 5):
    """
    Crée un retriever pour un namespace spécifique ou global
    
    Args:
        namespace: Namespace ciblé (None = tous les namespaces)
            Valeurs possibles :
            - "financial_reports" : Rapports SEC, données financières
            - "news" : Articles NewsAPI, Google RSS, communiqués
            - "macro_data" : Indicateurs macro (FRED, DBnomics, yfinance)
            - "startups" : Données Crunchbase, SIRENE
            - "social_signals" : Posts Bluesky
            - "facts" : Facts structurés (FACT-RAG)
            - "web_quarantine" : Sources web dynamiques (RAG agentic)
            - None : Recherche sur TOUS les namespaces
        k: Nombre de documents à récupérer
    
    Returns:
        Retriever configuré
    """
    vectorstore = PineconeVectorStore(
        index_name=INDEX_NAME,
        embedding=embeddings,
        namespace=namespace
    )
    
    retriever = vectorstore.as_retriever(
        search_type="similarity",
        search_kwargs={"k": k}
    )
    
    return retriever

# ═══════════════════════════════════════════════════════════════
# SECTION 3 : PROMPT ENGINEERING KPMG
# ═══════════════════════════════════════════════════════════════

"""
PROMPT STRUCTURÉ SELON LES EXIGENCES KPMG

Inspiré de vos notes (hackathon KPMG.pdf) :
✓ Assistant Intelligent de Veille Stratégique
✓ Citations OBLIGATOIRES avec source, date, fiabilité
✓ Réponse en prose (pas de bullet points par défaut)
✓ Indication si données manquantes ou payantes
✓ Capacité à demander des précisions

STRUCTURE :
1. Rôle et expertise
2. Instructions de citation
3. Format de réponse
4. Gestion des cas limites
"""

KPMG_PROMPT_TEMPLATE = """Vous êtes l'Assistant Intelligent de Veille Stratégique de KPMG Global Strategy Group.

Votre mission : Fournir des analyses de marché précises, sourcées et actionnables pour aider nos clients à prendre des décisions d'investissement éclairées.

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
RÈGLES DE CITATION (OBLIGATOIRES)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Pour CHAQUE information factuelle (chiffres, dates, faits) vous DEVEZ :
1. Citer la source exacte (ex: "SEC Filing 10-K d'Apple - 2024-01-15")
2. Indiquer le niveau de fiabilité :
   - ⭐⭐⭐ : Source primaire (SEC, rapport officiel, yfinance)
   - ⭐⭐ : Source secondaire fiable (NewsAPI, presse reconnue)
   - ⭐ : Source tertiaire (blogs, réseaux sociaux)
3. Préciser la date de l'information si critique

Format de citation : [Source | Fiabilité | Date]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
CONTEXTE DISPONIBLE
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

{context}

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
QUESTION DU CLIENT
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

{question}

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
INSTRUCTIONS DE RÉPONSE
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

0.
RÈGLE ABSOLUE DE FACTUALITÉ :
- Tu n’as PAS le droit d’utiliser tes connaissances générales
- Tu dois UNIQUEMENT utiliser les informations présentes dans le CONTEXTE
- Si une information n’est pas dans le CONTEXTE, tu dois dire explicitement qu’elle est indisponible
- Toute affirmation doit être associée à une source identifiable


1. STRUCTURE :
   - Répondez en prose fluide (paragraphes, pas de bullet points)
   - Organisez votre réponse de façon logique et narrative
   - Utilisez des transitions naturelles entre les idées

2. CONTENU :
   - Citez systématiquement vos sources (format ci-dessus)
   - Si une donnée est manquante : indiquez-le explicitement
   - Si une information nécessite un accès payant : précisez-le
   - Si le contexte est ambigu : demandez des précisions au client

3. TONE :
   - Professionnel mais accessible
   - Factuel et analytique
   - Confiant sur les données sourcées, prudent sur les spéculations

4. CAS LIMITES :
   - Si vous ne trouvez pas l'information : "Les données disponibles ne permettent pas de répondre à cette question. Sources consultées : [liste]. Je recommande [action]."
   - Si deux sources se contredisent : Mentionnez les deux et expliquez pourquoi
   - Si une entreprise est ambiguë : "J'ai identifié plusieurs entreprises nommées [X]. Pouvez-vous préciser : secteur, géographie, ou autre contexte ?"

5. Indique explicitement si une information provient :
  • de la base interne KPMG
  • d’une recherche externe automatique


━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
RÉPONSE ANALYTIQUE
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
"""

prompt = ChatPromptTemplate.from_template(KPMG_PROMPT_TEMPLATE)

print(" Prompt KPMG configuré")

# ═══════════════════════════════════════════════════════════════
# SECTION 4 : FORMATAGE DU CONTEXTE
# ═══════════════════════════════════════════════════════════════

"""
FORMATAGE DES DOCUMENTS RÉCUPÉRÉS

On enrichit le contexte avec :
- Source et type de document
- Date de publication/scraping
- Namespace d'origine
- Score de pertinence (si disponible)
"""

def format_docs(docs) -> str:
    """
    Formate les documents récupérés pour le prompt
    
    Args:
        docs: Documents LangChain
    
    Returns:
        String formaté avec métadonnées enrichies
    """
    formatted = []
    
    for i, doc in enumerate(docs, 1):
        metadata = doc.metadata
        
        # Construction de l'entrée
        entry = f"━━━ DOCUMENT {i} ━━━\n"
        entry += f"Source : {metadata.get('source', 'Unknown')}\n"
        entry += f"Type : {metadata.get('namespace', 'Unknown')}\n"
        
        # Date si disponible
        date_fields = ['filing_date', 'published_at', 'scrape_date', 'retrieval_date']
        for field in date_fields:
            if field in metadata:
                entry += f"Date : {metadata[field]}\n"
                break
        
        # URL si disponible
        if 'url' in metadata:
            entry += f"URL : {metadata['url']}\n"
        
        # Contenu
        entry += f"\nContenu :\n{doc.page_content}\n"
        
        formatted.append(entry)
    
    return "\n\n".join(formatted)

# ═══════════════════════════════════════════════════════════════
# SECTION 5 : CHAÎNE RAG COMPLÈTE
# ═══════════════════════════════════════════════════════════════

"""
ARCHITECTURE LCEL (LangChain Expression Language)

Pipeline : Retriever → Format Context → Prompt → LLM → Parse

RÉFÉRENCE :
https://python.langchain.com/docs/expression_language/
"""

def create_rag_chain(namespace: Optional[str] = None):
    """
    Crée une chaîne RAG complète
    
    Args:
        namespace: Namespace ciblé (None = tous)
    
    Returns:
        Chaîne RAG exécutable
    """
    retriever = get_retriever(namespace=namespace, k=5)
    
    rag_chain = (
        {
            "context": retriever | format_docs,
            "question": RunnablePassthrough()
        }
        | prompt
        | llm
        | StrOutputParser()
    )
    
    return rag_chain

# ═══════════════════════════════════════════════════════════════
# SECTION 6 : INTERFACE DE REQUÊTE
# ═══════════════════════════════════════════════════════════════

"""
FONCTIONS D'INTERFACE UTILISATEUR

Permettent d'interroger le système de différentes manières :
- Requête simple (tous namespaces)
- Requête ciblée (namespace spécifique)
- Requête multi-namespaces (comparaison)
"""

def query_veille(question: str, namespace: Optional[str] = None) -> str:
    """
    Interface principale de requête
    
    Args:
        question: Question de l'utilisateur
        namespace: Namespace ciblé (optionnel)
    
    Returns:
        Réponse formatée avec citations
    """
    print("\n" + "="*60)
    print(" ANALYSE EN COURS...")
    print("="*60)
    print(f"Question : {question}")
    
    if namespace:
        print(f"Namespace : {namespace}")
    else:
        print("Namespace : Tous (recherche globale)")
    
    print("="*60 + "\n")
    
    try:
                # 1. Récupération initiale (interne uniquement)
        retriever = get_retriever(namespace=namespace, k=5)
        docs = retriever.invoke(question)

        # 2. Jugement de suffisance
        context_ok = judge_context_sufficiency(
            question=question,
            docs=docs,
            llm=llm
        )

        final_docs = docs

        # 3. Fallback Web si nécessaire
    
        if not context_ok:
            print(" Contexte insuffisant → recherche externe activée")

            web_results = web_search_tool.run(question)
            web_docs = []

            for res in web_results:
                web_docs.append(
                    Document(
                        page_content=res["content"],
                        metadata={
                            "source": res.get("url"),
                            "origin": "web",
                            "confidence": score_source(res.get("url")),
                            "validated": False
                        }
                    )
                )

            # 4. Ingestion contrôlée (QUARANTAINE)
            vectorstore = PineconeVectorStore(
                index_name=INDEX_NAME,
                embedding=embeddings,
                namespace=WEB_QUARANTINE_NAMESPACE
            )

            # ════════════════════════════════════════════════════════════
            # AJOUT CONTRÔLÉ À PINECONE (AVEC ANTI-DOUBLONS)
            # ════════════════════════════════════════════════════════════

            pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
            index = pc.Index(INDEX_NAME)

            for doc in web_docs:
                content_hash = hashlib.md5(doc.page_content.encode('utf-8')).hexdigest()
                doc_id = f"web_{content_hash}"

            
                # Génération d'ID déterministe (même source = même ID)
                content_hash = hashlib.md5(doc.page_content.encode('utf-8')).hexdigest()
                vector_id = f"web_{content_hash}"
                
                # Récupérer l'embedding du document
                embedding = embeddings.embed_documents([doc.page_content])[0]
                
                # Métadonnées nettoyées
                clean_metadata = {
                    "source": doc.metadata.get("source", "")[:500],
                    "origin": "web",
                    "confidence": doc.metadata.get("confidence", 1),
                    "text": doc.page_content[:1000],
                    "validated": False,
                    "added_date": datetime.now().isoformat()
                }
                
                # Upsert (remplace si ID existe déjà)
                index.upsert(
                    vectors=[{
                        "id": vector_id,
                        "values": embedding,
                        "metadata": clean_metadata
                    }],
                    namespace=WEB_QUARANTINE_NAMESPACE
                )

            print(f" {len(web_docs)} sources web ajoutées au namespace quarantine")

            # Fusion du contexte
            final_docs = docs + web_docs

            has_web_sources = any(
            doc.metadata.get("origin") == "web" for doc in final_docs
        )

            has_reliable_web_sources = has_reliable_sources(final_docs)

            # Cas 1 — Aucune source fiable du tout → refus (normal)
            if not has_reliable_web_sources:
                return (
                    "⚠️ Information indisponible : aucune source fiable n’a été "
                    "identifiée dans la base interne ou via la recherche externe.\n\n"
                    "Conformément aux standards de gouvernance, "
                    "le système ne produit pas d’analyse factuelle non sourcée."
                )

            # Cas 2 — Sources web fiables trouvées → ON RÉPOND
            used_external_sources = has_web_sources



        # 5. Génération finale
        formatted_context = format_docs(final_docs)


        external_disclaimer = ""
        if used_external_sources:
            external_disclaimer = (
                "NOTE IMPORTANTE :\n"
                "Aucune information pertinente n’a été trouvée dans la base interne.\n"
                "Une recherche externe ciblée a donc été menée.\n\n"
            )

        response = (
            prompt
            | llm
            | StrOutputParser()
        ).invoke({
            "context": formatted_context,
            "question": external_disclaimer + question
        })


        return response

    
    except Exception as e:
        return f"Erreur lors de la requête : {e}"


def compare_namespaces(question: str, namespaces: List[str]) -> Dict[str, str]:
    """
    Compare les réponses de plusieurs namespaces
    
    Args:
        question: Question
        namespaces: Liste de namespaces à comparer
    
    Returns:
        Dictionnaire {namespace: réponse}
    """
    results = {}
    
    for ns in namespaces:
        print(f"\n Interrogation de '{ns}'...")
        results[ns] = query_veille(question, namespace=ns)
    
    return results

# ═══════════════════════════════════════════════════════════════
# SECTION 7 : EXEMPLES D'UTILISATION
# ═══════════════════════════════════════════════════════════════

"""
SCÉNARIOS DE DÉMONSTRATION KPMG

Ces exemples illustrent les capacités du système :
1. Analyse de marché
2. Due diligence d'entreprise
3. Détection de tendances
4. Analyse concurrentielle
"""

def demo_scenarios():
    """Démontre les capacités du système avec des cas réels"""
    
    print("\n" + " "*20)
    print(" DÉMONSTRATION RAG VEILLE KPMG")
    print(""*20 + "\n")
    
    scenarios = [
        {
            "titre": "1. ANALYSE FINANCIÈRE D'ENTREPRISE",
            "question": "Quelle est la capitalisation boursière actuelle d'Apple et son évolution ?",
            "namespace": "macro_data"
        },
        {
            "titre": "2. VEILLE ACTUALITÉS SECTEUR TECH",
            "question": "Quelles sont les dernières actualités concernant l'intelligence artificielle et la finance ?",
            "namespace": "news"
        },
        {
            "titre": "3. RECHERCHE GLOBALE (TOUS NAMESPACES)",
            "question": "Quels sont les principaux risques et opportunités pour les entreprises tech en 2024 ?",
            "namespace": None
        }
    ]
    
    for scenario in scenarios:
        print("\n" + "─"*60)
        print(f" {scenario['titre']}")
        print("─"*60)
        
        response = query_veille(
            question=scenario['question'],
            namespace=scenario['namespace']
        )
        
        print("\n RÉPONSE :\n")
        print(response)
        print("\n")


print("\n Système RAG KPMG opérationnel")
print(" Prêt pour l'interface Gradio (optionnel)")

 Modèles Mistral initialisés
 Prompt KPMG configuré

 Système RAG KPMG opérationnel
 Prêt pour l'interface Gradio (optionnel)


In [ ]:
"""
GRADIO INTERFACE - VERSION AVEC MISTRAL-SMALL (GRATUIT)
========================================================

Cette version utilise mistral-small au lieu de mistral-medium.
Performance légèrement inférieure mais GRATUIT et SUFFISANT pour votre démo.
"""

import os
import sys
import gradio as gr
from dotenv import load_dotenv

# ═══════════════════════════════════════════════════════════════
# AJOUT DU PATH POUR LES MODULES src/
# ═══════════════════════════════════════════════════════════════
src_path = os.path.join(os.getcwd(), "src")
if src_path not in sys.path:
    sys.path.insert(0, src_path)
    print(f"✅ Ajouté au path: {src_path}")

import kpmg_interface
import importlib
import analytics_viz
import facts_manager
import market_estimation_engine
import strategic_facts_service

from langchain_mistralai import ChatMistralAI, MistralAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# ═══════════════════════════════════════════════════════════════
# INITIALISATION
# ═══════════════════════════════════════════════════════════════

print("🔧 Initialisation de l'interface KPMG (mistral-small)...")

load_dotenv()

MISTRAL_API_KEY = os.getenv("MISTRAL_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
INDEX_NAME = "kpmg-veille"

if not MISTRAL_API_KEY or not PINECONE_API_KEY:
    raise ValueError("❌ Clés API manquantes dans .env")

print("✅ Variables d'environnement chargées")

# ═══════════════════════════════════════════════════════════════
# COMPOSANTS RAG
# ═══════════════════════════════════════════════════════════════

try:
    # Embeddings
    embeddings = MistralAIEmbeddings(
        model="mistral-embed",
        mistral_api_key=MISTRAL_API_KEY
    )
    print("✅ Embeddings initialisés")
    
    # Vector Store
    vectorstore = PineconeVectorStore(
        index_name=INDEX_NAME,
        embedding=embeddings,
        namespace="news"
    )
    
    retriever = vectorstore.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 3}
    )
    print("✅ Retriever configuré")
    
    # ╔═══════════════════════════════════════════════════════════╗
    # ║ CHANGEMENT CRITIQUE : mistral-medium → mistral-small    ║
    # ║                                                           ║
    # ║ Mistral-small est GRATUIT et suffisant pour du RAG      ║
    # ║ Performance : 85% de mistral-medium à coût zéro         ║
    # ╚═══════════════════════════════════════════════════════════╝
    
    llm = ChatMistralAI(
        model="mistral-small",  # ✅ MODÈLE GRATUIT
        temperature=0,
        mistral_api_key=MISTRAL_API_KEY
    )
    print("✅ LLM Mistral Small initialisé")
    
except Exception as e:
    print(f"❌ Erreur lors de l'initialisation : {e}")
    raise

# ═══════════════════════════════════════════════════════════════
# PROMPT KPMG
# ═══════════════════════════════════════════════════════════════

KPMG_PROMPT_TEMPLATE = """Vous êtes l'Assistant Intelligent de Veille Stratégique de KPMG Global Strategy Group.

Votre mission : Fournir des analyses de marché précises, sourcées et actionnables pour aider nos clients à prendre des décisions d'investissement éclairées.

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
RÈGLES DE CITATION (OBLIGATOIRES)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Pour CHAQUE information factuelle (chiffres, dates, faits) vous DEVEZ :
1. Citer la source exacte (ex: "https://www.apple.com - 2024-01-15")
2. Indiquer le niveau de fiabilité :
   - *** : Source primaire (SEC, rapport officiel, yfinance)
   - ** : Source secondaire fiable (NewsAPI, presse reconnue)
   - * : Source tertiaire (blogs, réseaux sociaux)
3. Préciser la date de l'information si critique

Format de citation : [Source au format URL | Fiabilité | Date]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
CONTEXTE DISPONIBLE
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

{context}

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
QUESTION DU CLIENT
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

{question}

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
INSTRUCTIONS DE RÉPONSE
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

1. STRUCTURE :
   - Répondez en prose fluide (paragraphes, pas de bullet points)
   - Organisez votre réponse de façon logique et narrative
   - Utilisez des transitions naturelles entre les idées

2. CONTENU :
   - Citez systématiquement vos sources (format ci-dessus)
   - Si une donnée est manquante : indiquez-le explicitement
   - Si une information nécessite un accès payant : précisez-le
   - Si le contexte est ambigu : demandez des précisions au client

3. TONE :
   - Professionnel mais accessible
   - Factuel et analytique
   - Confiant sur les données sourcées, prudent sur les spéculations

4. CAS LIMITES :
   - Si vous ne trouvez pas l'information : "Les données disponibles ne permettent pas de répondre à cette question. Sources consultées : [liste]. Je recommande [action]."
   - Si deux sources se contredisent : Mentionnez les deux et expliquez pourquoi
   - Si une entreprise est ambiguë : "J'ai identifié plusieurs entreprises nommées [X]. Pouvez-vous préciser : secteur, géographie, ou autre contexte ?"

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
RÉPONSE ANALYTIQUE
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
"""

prompt = ChatPromptTemplate.from_template(KPMG_PROMPT_TEMPLATE)
print("✅ Prompt configuré")

# ═══════════════════════════════════════════════════════════════
# CHAÎNE RAG
# ═══════════════════════════════════════════════════════════════

def format_docs(docs):
    """Formate les documents récupérés"""
    if not docs:
        return "Aucun document pertinent trouvé."
    
    formatted = []
    for i, doc in enumerate(docs, 1):
        source = doc.metadata.get('source', 'Unknown')
        date = doc.metadata.get('published_at', doc.metadata.get('scrape_date', 'N/A'))
        content = doc.page_content[:500]
        
        formatted.append(f"[Document {i} - Source: {source} - Date: {date}]\n{content}")
    
    return "\n\n".join(formatted)

rag_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)

print("✅ Chaîne RAG construite")

# ═══════════════════════════════════════════════════════════════
# FONCTION STREAMING
# ═══════════════════════════════════════════════════════════════

def stream_kpmg_response(message, history):
    """Génère la réponse de manière progressive"""
    try:
        partial_message = ""
        
        for chunk in rag_chain.stream(message):
            partial_message += chunk
            yield partial_message
    
    except Exception as e:
        error_msg = f"""❌ Erreur lors de la recherche :
        
Détails : {str(e)}

Suggestions :
1. Vérifiez que l'index Pinecone contient des données
2. Testez avec une question plus simple
3. Vérifiez les crédits Mistral sur console.mistral.ai"""
        
        yield error_msg

# ═══════════════════════════════════════════════════════════════
# LANCEMENT DE L'INTERFACE
# ═══════════════════════════════════════════════════════════════

# Recharger les modules pour prendre en compte les modifications
importlib.reload(facts_manager)
importlib.reload(market_estimation_engine)
importlib.reload(analytics_viz)
importlib.reload(kpmg_interface)
importlib.reload(strategic_facts_service)
print("✅ Modules rechargés à chaud !")

# Lance le dashboard KPMG
kpmg_interface.launch_dashboard(stream_kpmg_response)

 Initialisation de l'interface KPMG (mistral-small)...
 Variables d'environnement chargées
 Embeddings initialisés
 Retriever configuré
LLM Mistral Small initialisé
Prompt configuré
Chaîne RAG construite


/var/folders/tm/7kbwgcgd1_v72w5khby1lll00000gn/T/ipykernel_11037/711394503.py:238: UserWarning: The parameters have been moved from the Blocks constructor to the launch() method in Gradio 6.0: theme, css. Please pass these parameters to launch() instead.
  with gr.Blocks(theme=theme, css=custom_css, title="KPMG Analytics") as demo:



 LANCEMENT DE L'INTERFACE
* Running on local URL:  http://127.0.0.1:7861


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


* Running on public URL: https://adc8717e9cb04eee0e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



 Interface lancée avec succès !


In [33]:
"""
SCRIPT DE TEST COMPLET - SYSTÈME RAG KPMG
==========================================

Exécutez ce script pour diagnostiquer tous les problèmes potentiels.
Copier-coller dans une nouvelle cellule de notebook.
"""

import os
import sys
from dotenv import load_dotenv

# ═══════════════════════════════════════════════════════════════
# FONCTION DE TEST AVEC GESTION D'ERREURS
# ═══════════════════════════════════════════════════════════════

def run_test(test_name, test_func):
    """Exécute un test et affiche le résultat"""
    try:
        print(f"\n{'='*60}")
        print(f" TEST : {test_name}")
        print('='*60)
        result = test_func()
        print(f" {test_name} : RÉUSSI")
        return True, result
    except Exception as e:
        print(f" {test_name} : ÉCHOUÉ")
        print(f"   Erreur : {str(e)}")
        return False, None

# ═══════════════════════════════════════════════════════════════
# TEST 1 : ENVIRONNEMENT
# ═══════════════════════════════════════════════════════════════

def test_environment():
    """Vérifie les variables d'environnement"""
    load_dotenv()
    
    required_vars = {
        "MISTRAL_API_KEY": os.getenv("MISTRAL_API_KEY"),
        "PINECONE_API_KEY": os.getenv("PINECONE_API_KEY")
    }
    
    missing = [k for k, v in required_vars.items() if not v]
    
    if missing:
        raise ValueError(f"Variables manquantes : {', '.join(missing)}")
    
    for key, value in required_vars.items():
        masked = value[:10] + "..." if value else "None"
        print(f"   {key}: {masked}")
    
    return required_vars

# ═══════════════════════════════════════════════════════════════
# TEST 2 : PINECONE
# ═══════════════════════════════════════════════════════════════

def test_pinecone():
    """Vérifie l'état de l'index Pinecone"""
    from pinecone import Pinecone
    
    pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
    index = pc.Index("kpmg-veille")
    stats = index.describe_index_stats()
    
    print(f"   Index : kpmg-veille")
    print(f"   Total vecteurs : {stats.total_vector_count}")
    print(f"   Dimension : 1024")
    
    if stats.total_vector_count == 0:
        print("     ATTENTION : Aucune donnée indexée !")
        print("    Exécutez les Notebooks 2, 3 et 4")
        raise ValueError("Index vide")
    
    print(f"\n    Namespaces :")
    for ns, info in stats.namespaces.items():
        print(f"      - {ns}: {info.vector_count} vecteurs")
    
    return stats

# ═══════════════════════════════════════════════════════════════
# TEST 3 : MISTRAL EMBEDDINGS
# ═══════════════════════════════════════════════════════════════

def test_embeddings():
    """Teste la génération d'embeddings"""
    from langchain_mistralai import MistralAIEmbeddings
    
    embeddings = MistralAIEmbeddings(
        model="mistral-embed",
        mistral_api_key=os.getenv("MISTRAL_API_KEY")
    )
    
    # Test sur une phrase simple
    test_text = "Apple annonce de nouveaux produits"
    embedding = embeddings.embed_query(test_text)
    
    print(f"   Texte : '{test_text}'")
    print(f"   Dimension : {len(embedding)}")
    print(f"   Type : {type(embedding)}")
    print(f"   Premiers 5 valeurs : {embedding[:5]}")
    
    if len(embedding) != 1024:
        raise ValueError(f"Dimension incorrecte : {len(embedding)} (attendu : 1024)")
    
    return embeddings

# ═══════════════════════════════════════════════════════════════
# TEST 4 : MISTRAL LLM
# ═══════════════════════════════════════════════════════════════

def test_llm():
    """Teste le modèle LLM"""
    from langchain_mistralai import ChatMistralAI
    
    llm = ChatMistralAI(
        model="mistral-small",
        temperature=0,
        mistral_api_key=os.getenv("MISTRAL_API_KEY")
    )
    
    # Test simple
    response = llm.invoke("Réponds en un mot : quelle est la capitale de la France ?")
    
    print(f"   Modèle : mistral-medium")
    print(f"   Question : Capitale de la France ?")
    print(f"   Réponse : {response.content}")
    
    if "Paris" not in response.content:
        print("     Réponse inattendue, mais API fonctionne")
    
    return llm

# ═══════════════════════════════════════════════════════════════
# TEST 5 : RETRIEVER
# ═══════════════════════════════════════════════════════════════

def test_retriever():
    """Teste la recherche vectorielle"""
    from langchain_pinecone import PineconeVectorStore
    from langchain_mistralai import MistralAIEmbeddings
    
    embeddings = MistralAIEmbeddings(model="mistral-embed")
    
    vectorstore = PineconeVectorStore(
        index_name="kpmg-veille",
        embedding=embeddings,
        namespace="news"  # Utilise le namespace qui a des données
    )
    
    # Test de recherche
    query = "Apple dernières actualités"
    results = vectorstore.similarity_search(query, k=3)
    
    print(f"   Query : '{query}'")
    print(f"   Namespace : news")
    print(f"   Résultats : {len(results)} documents")
    
    if not results:
        raise ValueError("Aucun résultat trouvé - vérifiez l'indexation")
    
    print(f"\n    Premier résultat :")
    print(f"      Source : {results[0].metadata.get('source', 'Unknown')}")
    print(f"      Contenu : {results[0].page_content[:150]}...")
    
    return results

# ═══════════════════════════════════════════════════════════════
# TEST 6 : CHAÎNE RAG COMPLÈTE
# ═══════════════════════════════════════════════════════════════

def test_rag_chain():
    """Teste la chaîne RAG complète"""
    from langchain_mistralai import ChatMistralAI, MistralAIEmbeddings
    from langchain_pinecone import PineconeVectorStore
    from langchain_core.prompts import ChatPromptTemplate
    from langchain_core.runnables import RunnablePassthrough
    from langchain_core.output_parsers import StrOutputParser
    
    # Composants
    embeddings = MistralAIEmbeddings(model="mistral-embed")
    vectorstore = PineconeVectorStore(
        index_name="kpmg-veille",
        embedding=embeddings,
        namespace="news"
    )
    retriever = vectorstore.as_retriever(search_kwargs={"k": 2})
    llm = ChatMistralAI(model="mistral-medium", temperature=0)
    
    prompt = ChatPromptTemplate.from_template(
        "Contexte : {context}\n\nQuestion : {question}\n\nRéponse courte :"
    )
    
    # Chaîne
    def format_docs(docs):
        return "\n\n".join([d.page_content[:200] for d in docs])
    
    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )
    
    # Test
    question = "Quelles sont les dernières actualités ?"
    print(f"   Question : '{question}'")
    print(f"   Traitement...")
    
    response = rag_chain.invoke(question)
    
    print(f"\n    Réponse générée ({len(response)} caractères) :")
    print(f"   {response[:300]}...")
    
    return rag_chain

# ═══════════════════════════════════════════════════════════════
# TEST 7 : GRADIO (OPTIONNEL)
# ═══════════════════════════════════════════════════════════════

def test_gradio():
    """Vérifie que Gradio est installé"""
    import gradio as gr
    
    version = gr.__version__
    print(f"   Version : {version}")
    
    if version < "4.0.0":
        print("     Version ancienne détectée")
        print("   Recommandé : pip install --upgrade gradio")
    
    return gr

# ═══════════════════════════════════════════════════════════════
# EXÉCUTION DES TESTS
# ═══════════════════════════════════════════════════════════════

def main():
    """Exécute tous les tests dans l'ordre"""
    
    print("\n" + " "*20)
    print("   DIAGNOSTIC COMPLET DU SYSTÈME RAG KPMG")
    print(" "*20 + "\n")
    
    results = {}
    
    # Liste des tests à exécuter
    tests = [
        ("Variables d'environnement", test_environment),
        ("État de Pinecone", test_pinecone),
        ("Mistral Embeddings", test_embeddings),
        ("Mistral LLM", test_llm),
        ("Retriever", test_retriever),
        ("Chaîne RAG complète", test_rag_chain),
        ("Gradio", test_gradio)
    ]
    
    # Exécution
    for test_name, test_func in tests:
        success, result = run_test(test_name, test_func)
        results[test_name] = {"success": success, "result": result}
    
    # Rapport final
    print("\n" + "="*60)
    print(" RAPPORT FINAL")
    print("="*60)
    
    passed = sum(1 for r in results.values() if r["success"])
    total = len(results)
    
    print(f"\nTests réussis : {passed}/{total}")
    
    if passed == total:
        print("\n TOUS LES TESTS SONT PASSÉS !")
        print(" Votre système est prêt pour Gradio")
        print("\n Prochaine étape :")
        print("   Exécutez la cellule 'gradio_interface_fixed.py'")
    else:
        print("\n  CERTAINS TESTS ONT ÉCHOUÉ")
        print("\n Actions recommandées :")
        
        for test_name, result in results.items():
            if not result["success"]:
                print(f"\n {test_name} :")
                
                if "environnement" in test_name.lower():
                    print("   → Vérifiez votre fichier .env")
                    print("   → load_dotenv() doit être appelé")
                
                elif "pinecone" in test_name.lower():
                    print("   → Exécutez les Notebooks 2, 3, 4 pour indexer des données")
                    print("   → Vérifiez que l'index 'kpmg-veille' existe")
                
                elif "embeddings" in test_name.lower():
                    print("   → Vérifiez votre MISTRAL_API_KEY")
                    print("   → Testez manuellement : https://console.mistral.ai/")
                
                elif "llm" in test_name.lower():
                    print("   → Vérifiez les crédits de votre compte Mistral")
                    print("   → Essayez 'mistral-small' si 'medium' ne fonctionne pas")
                
                elif "retriever" in test_name.lower():
                    print("   → L'index est vide ou le namespace 'news' n'existe pas")
                    print("   → Réexécutez le Notebook 4 (indexation)")
                
                elif "rag" in test_name.lower():
                    print("   → Un composant précédent a échoué")
                    print("   → Corrigez les erreurs ci-dessus d'abord")
    
    print("\n" + "="*60)
    print("Diagnostic terminé")
    print("="*60 + "\n")
    
    return results

# ═══════════════════════════════════════════════════════════════
# LANCEMENT
# ═══════════════════════════════════════════════════════════════

if __name__ == "__main__":
    results = main()


                    
   DIAGNOSTIC COMPLET DU SYSTÈME RAG KPMG
                    


 TEST : Variables d'environnement
   MISTRAL_API_KEY: vd0pw28udW...
   PINECONE_API_KEY: pcsk_4PWXZ...
 Variables d'environnement : RÉUSSI

 TEST : État de Pinecone
   Index : kpmg-veille
   Total vecteurs : 12374
   Dimension : 1024

    Namespaces :
      - macro_data: 144 vecteurs
      - news: 1521 vecteurs
      - social_signals: 150 vecteurs
      - financial_reports: 10530 vecteurs
      - facts: 29 vecteurs
 État de Pinecone : RÉUSSI

 TEST : Mistral Embeddings
   Texte : 'Apple annonce de nouveaux produits'
   Dimension : 1024
   Type : <class 'list'>
   Premiers 5 valeurs : [-0.0162811279296875, 0.0140228271484375, 0.052886962890625, -0.0181427001953125, 0.04644775390625]
 Mistral Embeddings : RÉUSSI

 TEST : Mistral LLM
   Modèle : mistral-medium
   Question : Capitale de la France ?
   Réponse : Paris.
 Mistral LLM : RÉUSSI

 TEST : Retriever
   Query : 'Apple dernières actualités'
   Nam